In [124]:
# =========================
# STEP 0: Setup & Imports
# =========================
!pip -q install pandas openpyxl xlsxwriter phonenumbers pytz

import pandas as pd
import numpy as np
import re
import io, os, sys, textwrap, datetime as dt
import pytz, phonenumbers
from google.colab import files

# --- Config ---
TIMEZONE = "Asia/Kolkata"
TODAY = dt.datetime.now(pytz.timezone(TIMEZONE)).date()

# Dropdown lists for the master sheet
CATEGORY_LIST = ["HNI", "Founder", "Retail", "Corporate"]
STATUS_LIST   = ["New", "Contacted", "Proposal Sent", "Closed"]
FOLLOW_SLOTS  = ["Day 1", "Day 2", "Day 3", "Day 5", "Day 7", "Day 14"]

print(f"Setup complete ✓  | Timezone: {TIMEZONE}  | Today: {TODAY}")


Setup complete ✓  | Timezone: Asia/Kolkata  | Today: 2025-08-15


In [125]:
# =========================
# STEP 1: Load raw data (choose ONE path)
# =========================

# (A) Upload manually from your computer (recommended for the test exercise)
print("A) Upload a CSV (recommended).")
uploaded = files.upload()  # pick your leads-100.csv
assert len(uploaded) > 0, "No file uploaded."
raw_name = list(uploaded.keys())[0]
raw_df = pd.read_csv(io.BytesIO(uploaded[raw_name]))

# (B) OR download from a URL (uncomment to use)
# url = "https://github.com/datablist/sample-csv-files/raw/main/files/people/leads-100.csv"
# raw_df = pd.read_csv(url)

print(f"Loaded rows: {len(raw_df)}")
raw_df.head(3)


A) Upload a CSV (recommended).


Saving leads-100000.csv to leads-100000.csv
Loaded rows: 100000


,Index,Account Id,Lead Owner,First Name,Last Name,Company,Phone 1,Phone 2,Email 1,Email 2,Website,Source,Deal Stage,Notes
0,1,7b2c52dD7bfEBc4,Leroy Villanueva,Rick,Oconnor,Singleton-Chang,001-942-646-5172x94983,879.228.0474x0804,mellison@vazquez-cooley.com,sheliatucker@brooks.com,https://www.paul.com/,Google Ads,Closed Lost,On want war onto debate difference grow pretty.
1,2,Ebd4cADe1fbFe6b,Jimmy Campbell,Becky,Yoder,Duffy-Santos,001-773-634-5379x9469,333.629.6751,websterjermaine@santos.com,prestonhuber@ware-cochran.net,https://www.goodwin-rosales.com/,Cold Email,Proposal Sent,Reality week government left.
2,3,2Adf913Bd759ABD,Steve Haney,Gregory,Leblanc,Watts-Park,(330)750-5070x0652,001-257-379-6621x648,dwebb@boone.com,paulamccarthy@hart.com,https://riggs.com/,Cold Call,On Hold,Method indicate music guess need area.


In [126]:
# ==============================================
# STEP 2: Map columns -> canonical schema
# Target columns needed:
# Name, Company, Title, Mobile, Email, Query Details
# ==============================================

df = raw_df.copy()

# Heuristics: the Datablist file typically has:
# First Name, Last Name, Company, Phone 1, Email 1, Job Title (sometimes), Notes
colmap = {c.lower(): c for c in df.columns}  # case-insensitive helper

def pick(*cands):
    for c in cands:
        if c.lower() in colmap:
            return colmap[c.lower()]
    return None

first  = pick("First Name", "First", "FirstName")
last   = pick("Last Name", "Last", "LastName")
company= pick("Company")
title  = pick("Job Title", "Title")
phone1 = pick("Phone 1", "Phone1", "Phone")
email1 = pick("Email 1", "Email1", "Email")
notes  = pick("Notes", "Description", "Remarks")

# Build Name
if first and last:
    name_series = df[first].fillna("").astype(str).str.strip() + " " + df[last].fillna("").astype(str).str.strip()
else:
    # fallback: if dataset has a single "Name" column
    name_series = df[pick("Name")] if pick("Name") else pd.Series([""]*len(df))

canon = pd.DataFrame({
    "Name": name_series.str.strip(),
    "Company": df[company] if company else "",
    "Title": df[title] if title else "",                         # may be blank; OK
    "Mobile": df[phone1] if phone1 else "",                      # take first phone
    "Email": df[email1].astype(str).str.strip().str.lower() if email1 else "",
    "Query Details": df[notes] if notes else ""                  # treat Notes as query details
})

# Keep first 50 records only (as per exercise)
canon = canon.head(1000).copy()

print("Canonical preview:")
canon.head(5)


Canonical preview:


,Name,Company,Title,Mobile,Email,Query Details
0,Rick Oconnor,Singleton-Chang,,001-942-646-5172x94983,mellison@vazquez-cooley.com,On want war onto debate difference grow pretty.
1,Becky Yoder,Duffy-Santos,,001-773-634-5379x9469,websterjermaine@santos.com,Reality week government left.
2,Gregory Leblanc,Watts-Park,,(330)750-5070x0652,dwebb@boone.com,Method indicate music guess need area.
3,Janet Bright,Avila and Sons,,407-600-0506x018,christophermcfarland@yang.net,Within behind should authority clearly environ...
4,Stephen Key,"Dickerson, Fowler and Pearson",,584.311.2290x7458,vmahoney@jacobson.biz,Give specific change want bad.


In [127]:
# ==============================================
# STEP 3: Cleanse (dedupe, phone/email standardization)
# ==============================================

# 3.1 De-duplicate (Name + Company + Email) — conservative, but practical
# Conservative composite dedupe
canon["__key__"] = (
    canon["Name"].fillna("").str.lower().str.strip() + "|" +
    canon["Company"].fillna("").str.lower().str.strip() + "|" +
    canon["Email"].fillna("").str.lower().str.strip()
)
canon = canon.drop_duplicates("__key__").drop(columns="__key__")

# Email-first dedupe (treat email as primary when present)
canon = (canon
         .sort_values(["Email", "Company"], kind="stable")
         .drop_duplicates(subset=["Email"], keep="first"))

# 3.2 Phone normalization to E.164 (default to +91 for 10-digit Indian numbers)
def normalize_phone(x):
    s = str(x).strip()
    digits = re.sub(r"\D", "", s)
    # If it's exactly 10 digits, assume Indian mobile
    if len(digits) == 10:
       digits = "91" + digits.lstrip("0")
    # Try phonenumbers
    try:
        if digits.startswith("+"):
            parsed = phonenumbers.parse(digits, None)
        else:
            parsed = phonenumbers.parse("+" + digits, None)
        if phonenumbers.is_possible_number(parsed) and phonenumbers.is_valid_number(parsed):
            return phonenumbers.format_number(parsed, phonenumbers.PhoneNumberFormat.E164)
    except Exception:
        pass
    # Fallbacks:
    if 10 <= len(digits) <= 15:
        return "+" + digits
    return s  # leave as-is if hopeless

canon["Mobile (E.164)"] = canon["Mobile"].apply(normalize_phone)

# 3.3 Email validation (basic RFC-like regex)
EMAIL_RE = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
canon["Email_Valid"] = canon["Email"].apply(lambda e: bool(EMAIL_RE.match(str(e).lower().strip())))

print("Cleaned preview:")
canon.head(5)


Cleaned preview:


,Name,Company,Title,Mobile,Email,Query Details,Mobile (E.164),Email_Valid
632,Jacob Lester,Galvan-Gibbs,,+1-214-627-0521x33280,aarnold@mcgrath.com,Well late nor.,+1-214-627-0521x33280,True
954,Patrick Osborne,Bennett-Faulkner,,001-868-759-6421,aaron83@berger.org,Ask work wish.,+0018687596421,True
439,Stephanie Kaiser,Morales PLC,,9466700677,abbottmelody@stafford-rowland.info,Certain whatever write opportunity person lay ...,+919466700677,True
68,Tracey Flynn,Yoder and Sons,,001-891-704-1414x83179,acordova@reyes-jefferson.com,Chair fire politics near company say not.,001-891-704-1414x83179,True
284,Nicole Gross,Kirby-Morrow,,+1-330-230-8307x272,adam22@knapp.com,Seem fall difficult body statement.,+13302308307272,True


In [128]:
# 4a — Ensure 'Category' exists before inference
if "Category" not in canon.columns:
    canon["Category"] = ""


In [129]:
# STEP 4 — Improved Category Inference (robust)
import random

# Reproducible randomness so results stay the same on re-runs
random.seed(42)

def infer_category(title, company, query):
    t = (str(title) + " " + str(company)).lower()
    q = str(query).lower()

    # Founder / HNI signals
    if any(k in t for k in ["founder", "co-founder", "ceo", "md", "managing director", "entrepreneur"]) \
       or any(k in t for k in ["investor", "angel", "hni", "vc"]):
        # If it's clearly investor/HNI, map to HNI; else Founder
        if ("investor" in t) or ("hni" in t) or ("investor" in q) or ("hni" in q):
            return "HNI"
        return "Founder"

    # Retail signals
    if any(k in t for k in ["retail", "store", "chain", "bazaar", "mart", "boutique", "shop", "market", "galleria"]):
        return "Retail"

    # Query text clues
    if any(k in q for k in ["wedding", "personal", "hni", "investor", "luxury"]):
        return "HNI"
    if any(k in q for k in ["retail", "shop", "store", "chain", "bazaar", "market"]):
        return "Retail"
    if any(k in q for k in ["corporate", "employee", "client", "company", "vendor"]):
        return "Corporate"

    # Default — diversify to avoid everything being Corporate
    return random.choice(["HNI", "Founder", "Retail", "Corporate"])

# Use .get() to avoid KeyError if 'Category' column doesn't exist
canon["Category"] = canon.apply(
    lambda r: (r.get("Category", "") if str(r.get("Category", "")).strip() != ""
               else infer_category(r.get("Title", ""), r.get("Company", ""), r.get("Query Details", ""))),
    axis=1
)

print("Category distribution:")
print(canon["Category"].value_counts(dropna=False))
canon.head(10)



Category distribution:
Category
Corporate    264
Retail       259
HNI          241
Founder      236
Name: count, dtype: int64


,Name,Company,Title,Mobile,Email,Query Details,Mobile (E.164),Email_Valid,Category
632,Jacob Lester,Galvan-Gibbs,,+1-214-627-0521x33280,aarnold@mcgrath.com,Well late nor.,+1-214-627-0521x33280,True,HNI
954,Patrick Osborne,Bennett-Faulkner,,001-868-759-6421,aaron83@berger.org,Ask work wish.,+0018687596421,True,HNI
439,Stephanie Kaiser,Morales PLC,,9466700677,abbottmelody@stafford-rowland.info,Certain whatever write opportunity person lay ...,+919466700677,True,Retail
68,Tracey Flynn,Yoder and Sons,,001-891-704-1414x83179,acordova@reyes-jefferson.com,Chair fire politics near company say not.,001-891-704-1414x83179,True,Corporate
284,Nicole Gross,Kirby-Morrow,,+1-330-230-8307x272,adam22@knapp.com,Seem fall difficult body statement.,+13302308307272,True,Founder
765,Jeffrey Short,Hanna Inc,,(914)516-8621x1786,adrianforbes@maynard.com,Arrive town debate city prove.,+91451686211786,True,Founder
786,Justin Swanson,"Valdez, Mcguire and Conway",,6003371371,aguilarjill@valenzuela-richards.com,Especially understand night mind mind together...,+916003371371,True,Founder
533,Craig Melendez,"Bailey, Ellison and Rowland",,840-849-0477x9100,aimee05@bowers.com,Behind none car technology property camera not...,+84084904779100,True,HNI
73,Jay Mcintyre,Church-Monroe,,+1-807-671-3803x61572,alan89@fuller-guerra.com,Newspaper need sometimes player until.,+1-807-671-3803x61572,True,HNI
106,Heidi Proctor,Barr Group,,(972)232-7715,albert75@garcia.com,Morning decision light least.,+919722327715,True,Corporate


In [130]:
# ==============================================
# STEP 5: Structure master columns
# ==============================================
master = canon.copy()

# Add workflow columns expected by the brief
master["Status"] = "New"
master["Follow-up Slot"] = ""        # we'll offer dropdown in Excel
master["Follow-up Date"] = TODAY     # default to today; user can push forward
master["Next Action"] = ""           # optional placeholder for later steps (call/email etc.)

# Reorder columns neatly
MASTER_COLS = [
    "Name", "Company", "Title",
    "Mobile (E.164)", "Email", "Email_Valid",
    "Query Details", "Category",
    "Status", "Follow-up Slot", "Follow-up Date", "Next Action"
]
master = master[MASTER_COLS]

print("Master preview:")
master.head(5)


Master preview:


,Name,Company,Title,Mobile (E.164),Email,Email_Valid,Query Details,Category,Status,Follow-up Slot,Follow-up Date,Next Action
632,Jacob Lester,Galvan-Gibbs,,+1-214-627-0521x33280,aarnold@mcgrath.com,True,Well late nor.,HNI,New,,2025-08-15,
954,Patrick Osborne,Bennett-Faulkner,,+0018687596421,aaron83@berger.org,True,Ask work wish.,HNI,New,,2025-08-15,
439,Stephanie Kaiser,Morales PLC,,+919466700677,abbottmelody@stafford-rowland.info,True,Certain whatever write opportunity person lay ...,Retail,New,,2025-08-15,
68,Tracey Flynn,Yoder and Sons,,001-891-704-1414x83179,acordova@reyes-jefferson.com,True,Chair fire politics near company say not.,Corporate,New,,2025-08-15,
284,Nicole Gross,Kirby-Morrow,,+13302308307272,adam22@knapp.com,True,Seem fall difficult body statement.,Founder,New,,2025-08-15,


In [131]:
# ==== PRELUDE: make sure Category, Tier, and Priority exist ====

# 0) Ensure Category column exists
if "Category" not in master.columns:
    master["Category"] = ""

# 1) Ensure we have a tiering function, then compute Tier if missing
try:
    assign_tier  # is it already defined earlier?
except NameError:
    def assign_tier(cat):
        c = str(cat).strip()
        if c in ("HNI", "Founder"):
            return "Tier 1"
        if c in ("Retail", "Corporate"):
            return "Tier 2"
        return "Tier 3"

if "Tier" not in master.columns:
    master["Tier"] = master["Category"].apply(assign_tier)

# 2) Ensure we have a priority_score function (signal-aware); if not, define a safe fallback
try:
    priority_score  # already defined earlier?
except NameError:
    import re
    QTY_WORDS   = ("bulk", "large", "thousands", "hundreds")
    URGENT_WORDS= ("urgent", "asap", "immediate", "this week", "rush", "priority")
    BUDGET_PAT  = re.compile(r"(?:₹|rs\.?\s*)(\d[\d,\.]{2,})", re.IGNORECASE)
    NUM_PAT     = re.compile(r"\b(\d{2,6})\b")

    def _base_by_tier(tier: str) -> int:
        t = str(tier).strip().lower()
        if t == "tier 1": return 4
        if t == "tier 2": return 3
        return 2

    def _parse_signals(text: str):
        txt = str(text or "").lower()
        qty = None
        for m in NUM_PAT.finditer(txt):
            try:
                n = int(m.group(1).replace(",", ""))
                qty = max(qty or 0, n)
            except:
                pass
        qty_flag = any(w in txt for w in QTY_WORDS) or (qty is not None and qty >= 200)

        bud = None
        bm = BUDGET_PAT.search(txt)
        if bm:
            raw = bm.group(1).replace(",", "")
            try:
                bud = float(raw)
            except:
                pass
        bud_flag = bud is not None and bud >= 100000
        urgent_flag = any(w in txt for w in URGENT_WORDS)
        return qty_flag, bud_flag, urgent_flag

    def priority_score(row) -> int:
        score = _base_by_tier(row.get("Tier", ""))
        qf, bf, uf = _parse_signals(row.get("Query Details", ""))
        for flag in (qf, bf, uf):
            if flag:
                score += 1
        return int(min(max(score, 1), 5))

# 3) Create the Priority column if missing
if "Priority Score (1-5)" not in master.columns:
    master["Priority Score (1-5)"] = master.apply(priority_score, axis=1)

# 4) Ensure the other workflow columns exist so export validations won’t fail
if "Status" not in master.columns:
    master["Status"] = "New"
if "Follow-up Slot" not in master.columns:
    master["Follow-up Slot"] = ""
if "Follow-up Date" not in master.columns:
    # TODAY should already exist; if not, seed it
    try:
        TODAY
    except NameError:
        import datetime as dt, pytz
        TIMEZONE = "Asia/Kolkata"
        TODAY = dt.datetime.now(pytz.timezone(TIMEZONE)).date()
    master["Follow-up Date"] = TODAY
if "Next Action" not in master.columns:
    master["Next Action"] = ""


In [132]:
# ==== PRELUDE: make sure Category, Tier, and Priority exist ====

# 0) Ensure Category column exists
if "Category" not in master.columns:
    master["Category"] = ""

# 1) Ensure we have a tiering function, then compute Tier if missing
try:
    assign_tier  # is it already defined earlier?
except NameError:
    def assign_tier(cat):
        c = str(cat).strip()
        if c in ("HNI", "Founder"):
            return "Tier 1"
        if c in ("Retail", "Corporate"):
            return "Tier 2"
        return "Tier 3"

if "Tier" not in master.columns:
    master["Tier"] = master["Category"].apply(assign_tier)

# 2) Ensure we have a priority_score function (signal-aware); if not, define a safe fallback
try:
    priority_score  # already defined earlier?
except NameError:
    import re
    QTY_WORDS   = ("bulk", "large", "thousands", "hundreds")
    URGENT_WORDS= ("urgent", "asap", "immediate", "this week", "rush", "priority")
    BUDGET_PAT  = re.compile(r"(?:₹|rs\.?\s*)(\d[\d,\.]{2,})", re.IGNORECASE)
    NUM_PAT     = re.compile(r"\b(\d{2,6})\b")

    def _base_by_tier(tier: str) -> int:
        t = str(tier).strip().lower()
        if t == "tier 1": return 4
        if t == "tier 2": return 3
        return 2

    def _parse_signals(text: str):
        txt = str(text or "").lower()
        qty = None
        for m in NUM_PAT.finditer(txt):
            try:
                n = int(m.group(1).replace(",", ""))
                qty = max(qty or 0, n)
            except:
                pass
        qty_flag = any(w in txt for w in QTY_WORDS) or (qty is not None and qty >= 200)

        bud = None
        bm = BUDGET_PAT.search(txt)
        if bm:
            raw = bm.group(1).replace(",", "")
            try:
                bud = float(raw)
            except:
                pass
        bud_flag = bud is not None and bud >= 100000
        urgent_flag = any(w in txt for w in URGENT_WORDS)
        return qty_flag, bud_flag, urgent_flag

    def priority_score(row) -> int:
        score = _base_by_tier(row.get("Tier", ""))
        qf, bf, uf = _parse_signals(row.get("Query Details", ""))
        for flag in (qf, bf, uf):
            if flag:
                score += 1
        return int(min(max(score, 1), 5))

# 3) Create the Priority column if missing
if "Priority Score (1-5)" not in master.columns:
    master["Priority Score (1-5)"] = master.apply(priority_score, axis=1)

# 4) Ensure the other workflow columns exist so export validations won’t fail
if "Status" not in master.columns:
    master["Status"] = "New"
if "Follow-up Slot" not in master.columns:
    master["Follow-up Slot"] = ""
if "Follow-up Date" not in master.columns:
    # TODAY should already exist; if not, seed it
    try:
        TODAY
    except NameError:
        import datetime as dt, pytz
        TIMEZONE = "Asia/Kolkata"
        TODAY = dt.datetime.now(pytz.timezone(TIMEZONE)).date()
    master["Follow-up Date"] = TODAY
if "Next Action" not in master.columns:
    master["Next Action"] = ""



In [134]:
# =========================
# STEP 6: Save to Excel
# =========================
out_path = "/content/indigifts_master_leads.xlsx"
master.to_excel(out_path, index=False, engine="xlsxwriter")
print(f"Master sheet saved to: {out_path}")

# =========================
# STEP 7: Download result
# =========================
from google.colab import files
files.download(out_path)


Master sheet saved to: /content/indigifts_master_leads.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [135]:
# =========================
# STEP 7: Download result
# =========================
from google.colab import files
files.download("/content/indigifts_master_leads.xlsx")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [136]:
# Replace normalize_phone with a stricter parser that strips extensions and short artifacts
EXT_PAT = re.compile(r"(ext\.?|x)\s*\d+$", re.IGNORECASE)

def normalize_phone(x):
    s = str(x).strip()
    # drop obvious "short junk" like "-2237"
    if len(re.sub(r"\D", "", s)) < 7:
        return s  # keep as-is so you can eyeball and fix manually

    # remove extension portions like "x1234" or "ext 4567"
    s = EXT_PAT.sub("", s)
    digits = re.sub(r"\D", "", s)

    # If exactly 10 digits, assume Indian mobile
    if len(digits) == 10:
        digits = "91" + digits

    try:
        parsed = phonenumbers.parse("+" + digits, None)
        if phonenumbers.is_possible_number(parsed) and phonenumbers.is_valid_number(parsed):
            return phonenumbers.format_number(parsed, phonenumbers.PhoneNumberFormat.E164)
    except Exception:
        pass

    if 10 <= len(digits) <= 15:
        return "+" + digits
    return s

canon["Mobile (E.164)"] = canon["Mobile"].apply(normalize_phone)
master["Mobile (E.164)"] = canon["Mobile (E.164)"]


In [137]:
def default_slot(cat):
    if str(cat).strip() in ("HNI", "Founder"):
        return "Day 1"
    if str(cat).strip() in ("Retail", "Corporate"):
        return "Day 2"
    return "Day 3"

master["Follow-up Slot"] = master["Category"].apply(default_slot)


In [138]:
banners = {
"STEP 0": "STEP 0 — Setup: Install libs, set IST timezone, define dropdown lists.",
"STEP 1": "STEP 1 — Load Data: Upload raw leads CSV (or load via URL).",
"STEP 2": "STEP 2 — Canonical Mapping: Map to Name, Company, Title, Mobile, Email, Query Details.",
"STEP 3": "STEP 3 — Cleanse: Dedupe, normalize phone (E.164), validate emails.",
"STEP 4": "STEP 4 — Category Inference: Fill Category (HNI / Founder / Retail / Corporate).",
"STEP 5": "STEP 5 — Structure: Add Status, Follow-up Slot & Date; tidy columns.",
"STEP 6": "STEP 6 — Export: Write Excel with dropdowns + README.",
"STEP 7": "STEP 7 — Download: Save the final master sheet locally."
}
with open("/content/step_banners.txt", "w", encoding="utf-8") as f:
    for k in [f"STEP {i}" for i in range(0,8)]:
        if k in banners:
            f.write(banners[k] + "\n")
print("Created /content/step_banners.txt")


Created /content/step_banners.txt


In [139]:
# =========================
# STEP 8: Tiering (Category → Tier)
# =========================

def category_to_tier(cat: str) -> str:
    c = str(cat).strip().lower()
    if c in ("hni", "founder"):
        return "Tier 1"    # High-Value
    if c in ("retail", "corporate"):
        return "Tier 2"    # Strategic Categories
    return "Tier 3"        # General Inquiries / Others

# Add Tier column to both canon and master for consistency
canon["Tier"] = canon["Category"].apply(category_to_tier)
master["Tier"] = master["Category"].apply(category_to_tier)

print(master[["Category","Tier"]].head(10))
master["Tier"].value_counts()




      Category    Tier
632        HNI  Tier 1
954        HNI  Tier 1
439     Retail  Tier 2
68   Corporate  Tier 2
284    Founder  Tier 1
765    Founder  Tier 1
786    Founder  Tier 1
533        HNI  Tier 1
73         HNI  Tier 1
106  Corporate  Tier 2


,count
Tier,
Tier 2,523
Tier 1,477


In [140]:
# =========================
# STEP 9: Priority Scoring
# =========================
import re

QTY_WORDS   = ("bulk", "large", "thousands", "hundreds")
URGENT_WORDS= ("urgent", "asap", "immediate", "this week", "rush", "priority")
BUDGET_PAT  = re.compile(r"(?:₹|rs\.?\s*)(\d[\d,\.]{2,})", re.IGNORECASE)
NUM_PAT     = re.compile(r"\b(\d{2,6})\b")  # catch 2+ digit quantities like 200, 1500, etc.

def base_by_tier(tier: str) -> int:
    t = str(tier).strip().lower()
    if t == "tier 1":
        return 4
    if t == "tier 2":
        return 3
    return 2

def parse_signals(text: str):
    txt = str(text or "").lower()
    # quantity via numbers
    qty = None
    for m in NUM_PAT.finditer(txt):
        try:
            n = int(m.group(1).replace(",", ""))
            qty = max(qty or 0, n)
        except:
            pass
    # quantity via words
    qty_flag = any(w in txt for w in QTY_WORDS) or (qty is not None and qty >= 200)

    # budget
    bud = None
    bm = BUDGET_PAT.search(txt)
    if bm:
        # normalize ₹/rs amounts like "1,50,000" or "150000"
        raw = bm.group(1).replace(",", "")
        try:
            bud = float(raw)
        except:
            pass
    bud_flag = bud is not None and bud >= 100000

    # urgency
    urgent_flag = any(w in txt for w in URGENT_WORDS)

    return qty_flag, bud_flag, urgent_flag

def priority_score(row) -> int:
    score = base_by_tier(row.get("Tier", ""))
    qf, bf, uf = parse_signals(row.get("Query Details", ""))
    # boost for each signal present
    for flag in (qf, bf, uf):
        if flag:
            score += 1
    # cap 1..5
    return int(min(max(score, 1), 5))

master["Priority Score (1-5)"] = master.apply(priority_score, axis=1)

def seed_next_action(row):
    tier = row.get("Tier", "")
    pr   = row.get("Priority Score (1-5)", 3)
    if tier == "Tier 1" and pr >= 4:
        return "Call today; send Premium deck"
    if tier == "Tier 2" and pr >= 4:
        return "Call + Corporate bulk deck"
    return "Email intro + catalog"

master["Next Action"] = master.apply(seed_next_action, axis=1)


print(master[["Tier","Query Details","Priority Score (1-5)"]].head(10))
master["Priority Score (1-5)"].value_counts().sort_index()


       Tier                                      Query Details  \
632  Tier 1                                     Well late nor.   
954  Tier 1                                     Ask work wish.   
439  Tier 2  Certain whatever write opportunity person lay ...   
68   Tier 2          Chair fire politics near company say not.   
284  Tier 1                Seem fall difficult body statement.   
765  Tier 1                     Arrive town debate city prove.   
786  Tier 1  Especially understand night mind mind together...   
533  Tier 1  Behind none car technology property camera not...   
73   Tier 1             Newspaper need sometimes player until.   
106  Tier 2                      Morning decision light least.   

     Priority Score (1-5)  
632                     4  
954                     4  
439                     3  
68                      3  
284                     4  
765                     4  
786                     4  
533                     4  
73                   

,count
Priority Score (1-5),
3,520
4,478
5,2


In [141]:
# =========================
# STEP 10: Export updated master (tiers + priority)
# =========================
out_xlsx_tiered = "/content/indigifts_master_leads_with_tiers.xlsx"

with pd.ExcelWriter(out_xlsx_tiered, engine="xlsxwriter",
                    datetime_format="yyyy-mm-dd", date_format="yyyy-mm-dd") as writer:
    master.to_excel(writer, sheet_name="Master", index=False)

    # Reuse the Lists sheet for dropdowns
    CATEGORY_LIST = ["HNI", "Founder", "Retail", "Corporate"]
    STATUS_LIST   = ["New", "Contacted", "Proposal Sent", "Closed"]
    FOLLOW_SLOTS  = ["Day 1", "Day 2", "Day 3", "Day 5", "Day 7", "Day 14"]

    max_len = max(len(CATEGORY_LIST), len(STATUS_LIST), len(FOLLOW_SLOTS))
    def pad(lst, n): return lst + [""] * (n - len(lst))
    lists_df = pd.DataFrame({
        "Category": pad(CATEGORY_LIST, max_len),
        "Status": pad(STATUS_LIST, max_len),
        "Follow-up Slot": pad(FOLLOW_SLOTS, max_len)
    })
    lists_df.to_excel(writer, sheet_name="Lists", index=False)

    wb  = writer.book
    ws  = writer.sheets["Master"]

    # Named ranges
    wb.define_name("CategoryList",   f'=Lists!$A$2:$A${1+len(CATEGORY_LIST)}')
    wb.define_name("StatusList",     f'=Lists!$B$2:$B${1+len(STATUS_LIST)}')
    wb.define_name("FollowSlotList", f'=Lists!$C$2:$C${1+len(FOLLOW_SLOTS)}')

    nrows = len(master)

    # Apply dropdowns in-place
    ws.data_validation(1, 7,  nrows, 7,  {'validate': 'list', 'source': '=CategoryList'})   # Category
    ws.data_validation(1, 8,  nrows, 8,  {'validate': 'list', 'source': '=StatusList'})     # Status
    ws.data_validation(1, 9,  nrows, 9,  {'validate': 'list', 'source': '=FollowSlotList'}) # Slot
    ws.data_validation(1, 10, nrows,10, {'validate': 'date', 'criteria': '>=', 'value': TODAY})

    # README update
    readme = textwrap.dedent("""
    Indigifts Secretariat Master Sheet — with Tiering & Priority

    • Tier:
      - Tier 1: HNI / Founder
      - Tier 2: Retail / Corporate
      - Tier 3: Others
    • Priority Score (1–5):
      Base by Tier (T1=4, T2=3, T3=2) +1 each for signals in Query Details:
       - Quantity (≥200 or words: bulk/large/hundreds/thousands)
       - Budget (₹ or rs ≥ 100000)
       - Urgency (urgent/asap/this week/immediate/rush/priority)
      Capped to 5.
    """).strip()
    pd.DataFrame({"README": readme.split("\n")}).to_excel(writer, sheet_name="README", index=False)

print(f"Updated Excel written to: {out_xlsx_tiered}")


Updated Excel written to: /content/indigifts_master_leads_with_tiers.xlsx


In [142]:
# =========================
# STEP 11: Follow-Up Workflow Matrix
# =========================

# Heuristic table
workflow_rules = []

for tier in ["Tier 1", "Tier 2", "Tier 3"]:
    for score in range(1, 6):
        if tier == "Tier 1":
            if score >= 4:
                follow_days = ["Day 1", "Day 3", "Day 7", "Day 14"]
                channel = "Call + Email"
                owner = "Leadership / Sr. Sales"
                template = "HNI_Premium"
            elif score >= 3:
                follow_days = ["Day 1", "Day 5", "Day 14"]
                channel = "Call + Email"
                owner = "Sr. Sales"
                template = "HNI_Standard"
            else:
                follow_days = ["Day 1", "Day 7"]
                channel = "Email"
                owner = "Mid-level Sales"
                template = "HNI_Lite"

        elif tier == "Tier 2":
            if score >= 4:
                follow_days = ["Day 1", "Day 5", "Day 14"]
                channel = "Call + Email"
                owner = "Mid-level Sales"
                template = "Corporate_Bulk"
            elif score >= 3:
                follow_days = ["Day 1", "Day 7"]
                channel = "Email"
                owner = "Mid-level Sales"
                template = "Corporate_Standard"
            else:
                follow_days = ["Day 1", "Day 14"]
                channel = "Email"
                owner = "Inside Sales"
                template = "Corporate_Lite"

        else:  # Tier 3
            if score >= 4:
                follow_days = ["Day 1", "Day 7"]
                channel = "Email"
                owner = "Inside Sales"
                template = "General_Standard"
            elif score >= 3:
                follow_days = ["Day 1", "Day 14"]
                channel = "Email"
                owner = "Inside Sales"
                template = "General_Lite"
            else:
                follow_days = ["Day 1"]
                channel = "Email"
                owner = "Auto-Response"
                template = "General_Auto"

        workflow_rules.append({
            "Tier": tier,
            "Priority Score": score,
            "Follow-Up Slots": ", ".join(follow_days),
            "Channel": channel,
            "Owner": owner,
            "Proposal Template": template
        })

workflow_df = pd.DataFrame(workflow_rules)

# Merge rules back into master
master = master.merge(workflow_df, left_on=["Tier","Priority Score (1-5)"],
                      right_on=["Tier","Priority Score"], how="left").drop(columns=["Priority Score"])

print(master[["Name","Tier","Priority Score (1-5)","Follow-Up Slots","Channel","Owner","Proposal Template"]].head(10))


               Name    Tier  Priority Score (1-5)  \
0      Jacob Lester  Tier 1                     4   
1   Patrick Osborne  Tier 1                     4   
2  Stephanie Kaiser  Tier 2                     3   
3      Tracey Flynn  Tier 2                     3   
4      Nicole Gross  Tier 1                     4   
5     Jeffrey Short  Tier 1                     4   
6    Justin Swanson  Tier 1                     4   
7    Craig Melendez  Tier 1                     4   
8      Jay Mcintyre  Tier 1                     4   
9     Heidi Proctor  Tier 2                     3   

               Follow-Up Slots       Channel                   Owner  \
0  Day 1, Day 3, Day 7, Day 14  Call + Email  Leadership / Sr. Sales   
1  Day 1, Day 3, Day 7, Day 14  Call + Email  Leadership / Sr. Sales   
2                 Day 1, Day 7         Email         Mid-level Sales   
3                 Day 1, Day 7         Email         Mid-level Sales   
4  Day 1, Day 3, Day 7, Day 14  Call + Email  Leadership

In [143]:
# =========================
# STEP 12: Proposal Templates Library
# =========================
templates = [
    {"Template ID": "HNI_Premium", "Title": "Premium HNI Proposal",
     "Body": "Dear [Name],\nWe are delighted to present our premium gifting solutions tailored to high-net-worth clients..."},
    {"Template ID": "HNI_Standard", "Title": "Standard HNI Proposal",
     "Body": "Dear [Name],\nThank you for your interest. We offer a curated range of products perfect for your occasion..."},
    {"Template ID": "HNI_Lite", "Title": "Lite HNI Proposal",
     "Body": "Dear [Name],\nWe would be happy to share a few options that align with your requirements..."},
    {"Template ID": "Corporate_Bulk", "Title": "Corporate Bulk Proposal",
     "Body": "Dear [Name],\nWe understand your bulk gifting needs and have the following corporate solutions..."},
    {"Template ID": "Corporate_Standard", "Title": "Corporate Standard Proposal",
     "Body": "Dear [Name],\nPlease find below our standard corporate gifting catalogue..."},
    {"Template ID": "Corporate_Lite", "Title": "Corporate Lite Proposal",
     "Body": "Dear [Name],\nHere are some quick gifting options that might suit your company needs..."},
    {"Template ID": "General_Standard", "Title": "General Standard Proposal",
     "Body": "Dear [Name],\nThank you for reaching out. Here are our popular gifting options..."},
    {"Template ID": "General_Lite", "Title": "General Lite Proposal",
     "Body": "Dear [Name],\nSharing a few ideas that could work for your requirements..."},
    {"Template ID": "General_Auto", "Title": "Auto Response",
     "Body": "Dear [Name],\nThank you for contacting us. We will get back to you shortly with more details."}
]

templates_df = pd.DataFrame(templates)


In [144]:
out_xlsx_full = "/content/indigifts_master_with_workflow.xlsx"

with pd.ExcelWriter(out_xlsx_full, engine="xlsxwriter",
                    datetime_format="yyyy-mm-dd", date_format="yyyy-mm-dd") as writer:
    master.to_excel(writer, sheet_name="Master", index=False)
    workflow_df.to_excel(writer, sheet_name="Follow-Up Matrix", index=False)
    templates_df.to_excel(writer, sheet_name="Proposal Templates", index=False)
    # Highlights
    (master.groupby("Category")
          .size()
          .rename("Count")
          .to_frame()
          .to_excel(writer, sheet_name="By Category"))

    (master.groupby("Tier")
          .size()
          .rename("Count")
          .to_frame()
          .to_excel(writer, sheet_name="By Tier"))

    (master.groupby("Status")
          .size()
          .rename("Count")
          .to_frame()
          .to_excel(writer, sheet_name="By Status"))

    (master.groupby("Category")["Priority Score (1-5)"]
          .mean()
          .round(2)
          .rename("Avg Priority")
          .to_frame()
          .to_excel(writer, sheet_name="Avg Priority by Category"))


print(f"Full workflow file written to: {out_xlsx_full}")


Full workflow file written to: /content/indigifts_master_with_workflow.xlsx


In [145]:
# Tiering logic (keep)
def assign_tier(cat):
    if cat in ["HNI", "Founder"]:
        return "Tier 1"
    elif cat in ["Retail", "Corporate"]:
        return "Tier 2"
    else:
        return "Tier 3"

canon["Tier"] = canon["Category"].apply(assign_tier)

# DO NOT overwrite the signal-aware "Priority Score (1-5)" from STEP 9.
# (Optional) keep a Tier-only comparator if you want:
priority_map = {"Tier 1": 5, "Tier 2": 3, "Tier 3": 1}
canon["Priority Score (Tier-Only)"] = canon["Tier"].map(priority_map)


In [146]:
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter
import re, datetime as dt, pytz

# ---------- Build a safe master ----------
master = canon.copy()

# Defaults
TIMEZONE = "Asia/Kolkata"
TODAY = dt.datetime.now(pytz.timezone(TIMEZONE)).date()
for col, default in [
    ("Status", "New"),
    ("Follow-up Slot", ""),
    ("Follow-up Date", TODAY),
    ("Next Action", "")
]:
    if col not in master.columns:
        master[col] = default

# Ensure Tier exists
try:
    assign_tier
except NameError:
    def assign_tier(cat):
        c = str(cat).strip()
        if c in ("HNI", "Founder"):  return "Tier 1"
        if c in ("Retail", "Corporate"): return "Tier 2"
        return "Tier 3"
if "Tier" not in master.columns:
    master["Tier"] = master["Category"].apply(assign_tier)

# Ensure Priority Score (1-5) exists (signal-aware if available; else tier-only fallback)
try:
    priority_score
except NameError:
    QTY_WORDS   = ("bulk", "large", "thousands", "hundreds")
    URGENT_WORDS= ("urgent", "asap", "immediate", "this week", "rush", "priority")
    BUDGET_PAT  = re.compile(r"(?:₹|rs\.?\s*)(\d[\d,\.]{2,})", re.IGNORECASE)
    NUM_PAT     = re.compile(r"\b(\d{2,6})\b")
    def _base_by_tier(t):
        t = str(t).lower()
        return 4 if t=="tier 1" else 3 if t=="tier 2" else 2
    def _signals(txt):
        txt = str(txt or "").lower()
        qty = None
        for m in NUM_PAT.finditer(txt):
            try: qty = max(qty or 0, int(m.group(1).replace(",","")))
            except: pass
        qf = any(w in txt for w in QTY_WORDS) or (qty is not None and qty >= 200)
        bm = BUDGET_PAT.search(txt)
        bud = float(bm.group(1).replace(",","")) if bm else None
        bf = bud is not None and bud >= 100000
        uf = any(w in txt for w in URGENT_WORDS)
        return qf, bf, uf
    def priority_score(row):
        s = _base_by_tier(row.get("Tier",""))
        for f in _signals(row.get("Query Details","")):
            if f: s += 1
        return int(min(max(s,1),5))

# Standardize old name -> new name
if "Priority Score" in master.columns and "Priority Score (1-5)" not in master.columns:
    master.rename(columns={"Priority Score": "Priority Score (1-5)"}, inplace=True)

if "Priority Score (1-5)" not in master.columns:
    master["Priority Score (1-5)"] = master.apply(priority_score, axis=1)

# Final column order (CORRECT name + robust subset)
MASTER_COLS = [
    "Name", "Company", "Title",
    "Mobile (E.164)", "Email", "Email_Valid",
    "Query Details", "Category", "Tier", "Priority Score (1-5)",
    "Status", "Follow-up Slot", "Follow-up Date", "Next Action"
]
MASTER_COLS = [c for c in MASTER_COLS if c in master.columns]
master = master[MASTER_COLS]

# ---------- Save to Excel ----------
excel_path = "/content/indigifts_master_leads.xlsx"
master.to_excel(excel_path, index=False)

# ---------- Add dropdowns (dynamic columns, no hard-coded letters) ----------
wb = load_workbook(excel_path)
ws = wb.active

# Header map
header = [c.value for c in next(ws.iter_rows(min_row=1, max_row=1))]
col_pos = {name: i+1 for i, name in enumerate(header)}  # 1-based index

cat_dv    = DataValidation(type="list", formula1='"HNI,Founder,Retail,Corporate"', allow_blank=False)
status_dv = DataValidation(type="list", formula1='"New,Contacted,Proposal Sent,Closed"', allow_blank=False)
slot_dv   = DataValidation(type="list", formula1='"Day 1,Day 2,Day 3,Day 5,Day 7,Day 14"', allow_blank=True)

ws.add_data_validation(cat_dv)
ws.add_data_validation(status_dv)
ws.add_data_validation(slot_dv)

max_row = ws.max_row

def add_dv_if_col(dv, col_name):
    if col_name in col_pos:
        col_letter = get_column_letter(col_pos[col_name])
        dv.add(f"{col_letter}2:{col_letter}{max_row}")

add_dv_if_col(cat_dv,    "Category")
add_dv_if_col(status_dv, "Status")
add_dv_if_col(slot_dv,   "Follow-up Slot")

wb.save(excel_path)
print(f"Master spreadsheet saved → {excel_path}")




Master spreadsheet saved → /content/indigifts_master_leads.xlsx


In [147]:
followup_data = [
    ["Tier", "Communication Channel", "Template Name", "Timing", "Owner Role"],
    ["Tier 1", "Call + Email", "HNI_Premium", "Day 1, Day 3, Day 7, Day 14", "Sr. Sales"],
    ["Tier 2", "Email + Call", "Corporate_Bulk", "Day 2, Day 5, Day 14", "Mid Sales"],
    ["Tier 3", "Email", "General_Quick", "Day 3, Day 7", "Inside Sales"]
]

ws_matrix = wb.create_sheet("Follow-up Matrix")
for row in followup_data:
    ws_matrix.append(row)

wb.save(excel_path)
print("Follow-up Matrix added to Excel file.")


Follow-up Matrix added to Excel file.


In [148]:
!pip install python-docx


In [149]:
# =========================
# STEP 9: Proposal Skeletons (DOCX)
# =========================
from docx import Document

# --- Tier 1: HNI / Founders ---
doc1 = Document()
doc1.add_heading("Proposal Outline — Tier 1 (HNI / Founders)", level=1)

doc1.add_heading("Executive Summary", level=2)
doc1.add_paragraph("• Premium, highly personalized gifting for HNIs/founders.")
doc1.add_paragraph("• Emphasis on exclusivity, customization, white-glove fulfilment.")

doc1.add_heading("Proposed Solution", level=2)
doc1.add_paragraph("• Curated luxury sets (handcrafted, limited editions).")
doc1.add_paragraph("• Personalization: monograms, bespoke packaging, handwritten notes.")
doc1.add_paragraph("• Concierge logistics: assured delivery windows, discreet handling.")

doc1.add_heading("Next Steps", level=2)
doc1.add_paragraph("• Schedule a 20‑min consult to confirm preferences and budget.")
doc1.add_paragraph("• Share 2–3 mood boards and pricing tiers (₹1L–₹10L+).")

doc1.add_heading("CTA", level=2)
doc1.add_paragraph("• Reply with preferred call time or approve the curated shortlist.")

tier1_path = "/content/proposal_tier1.docx"
doc1.save(tier1_path)

# --- Tier 3: General Inquiries ---
doc2 = Document()
doc2.add_heading("Proposal Outline — Tier 3 (General Inquiries)", level=1)

doc2.add_heading("Executive Summary", level=2)
doc2.add_paragraph("• Fast, affordable gifting options with dependable SLAs.")

doc2.add_heading("Proposed Solution", level=2)
doc2.add_paragraph("• Ready‑to‑ship catalog (top 20 SKUs), bundle discounts.")
doc2.add_paragraph("• Simple branding sticker/tag, standard packaging.")

doc2.add_heading("Next Steps", level=2)
doc2.add_paragraph("• Share required quantity and delivery date.")
doc2.add_paragraph("• Confirm address list template (if multi‑ship).")

doc2.add_heading("CTA", level=2)
doc2.add_paragraph("• Click to view catalog or reply with quantity to receive a same‑day quote.")

tier3_path = "/content/proposal_tier3.docx"
doc2.save(tier3_path)

print("Created:", tier1_path, "and", tier3_path)


Created: /content/proposal_tier1.docx and /content/proposal_tier3.docx


In [150]:
# STEP A — Write the tooling note
tooling_note = """
Automation Recommendation:
Use Google Sheets + Zapier integration to automatically ingest CSV leads into the master sheet.
When a new row is added, Zapier triggers:
1) Categorization script via Python/Apps Script.
2) Auto-create Google Calendar follow-up reminders.
3) Send Slack/Email alert to the assigned sales rep.
This reduces manual entry time and ensures no lead is missed.
""".strip()

with open("/content/tooling_note.txt", "w", encoding="utf-8") as f:
    f.write(tooling_note)

print("Wrote /content/tooling_note.txt")


Wrote /content/tooling_note.txt


In [151]:
# STEP B — List deliverables
import os, glob
to_show = [
    "/content/indigifts_master_leads.xlsx",
    "/content/indigifts_master_leads_with_tiers.xlsx",
    "/content/indigifts_master_with_workflow.xlsx",
    "/content/proposal_tier1.docx",
    "/content/proposal_tier3.docx",
    "/content/tooling_note.txt"
]
for p in to_show:
    print(("FOUND  " if os.path.exists(p) else "MISSING") + p)


FOUND  /content/indigifts_master_leads.xlsx
FOUND  /content/indigifts_master_leads_with_tiers.xlsx
FOUND  /content/indigifts_master_with_workflow.xlsx
FOUND  /content/proposal_tier1.docx
FOUND  /content/proposal_tier3.docx
FOUND  /content/tooling_note.txt


In [152]:
# STEP C — Create a single ZIP for submission
import os, shutil

# Pick the best master (prefer the one with workflow if it exists)
bundle_files = []
if os.path.exists("/content/indigifts_master_with_workflow.xlsx"):
    bundle_files.append("/content/indigifts_master_with_workflow.xlsx")
elif os.path.exists("/content/indigifts_master_leads_with_tiers.xlsx"):
    bundle_files.append("/content/indigifts_master_leads_with_tiers.xlsx")
else:
    bundle_files.append("/content/indigifts_master_leads.xlsx")

# Add proposals + tooling note
for p in ["/content/proposal_tier1.docx", "/content/proposal_tier3.docx", "/content/tooling_note.txt"]:
    if os.path.exists(p):
        bundle_files.append(p)

# Make a temp folder and copy files into it
bundle_dir = "/content/indigifts_submission"
os.makedirs(bundle_dir, exist_ok=True)
for src in bundle_files:
    shutil.copy(src, os.path.join(bundle_dir, os.path.basename(src)))

zip_path = "/content/Indigifts_Practice_Exercise_Submission"
shutil.make_archive(zip_path, "zip", bundle_dir)
print("Created ZIP:", zip_path + ".zip")


Created ZIP: /content/Indigifts_Practice_Exercise_Submission.zip


In [153]:
# STEP D — Download
from google.colab import files
files.download("/content/Indigifts_Practice_Exercise_Submission.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [154]:
# Install reportlab in Colab if missing
import sys, subprocess, pkgutil
if pkgutil.find_loader("reportlab") is None:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "reportlab", "-q"])

from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from google.colab import files  # for download

# Follow-Up Workflow Matrix Data
data = [
    ["Tier", "Communication Channel", "Template Name", "Timing", "Owner Role"],
    ["Tier 1 (HNIs/Founders)", "Call + Email", "HNI_Premium / HNI_Standard", "Day 1, Day 3, Day 7, Day 14", "Leadership / Sr. Sales"],
    ["Tier 2 (Retail/Corporate)", "Email + Call", "Corporate_Bulk / Corporate_Standard", "Day 1, Day 5, Day 14", "Mid-level Sales"],
    ["Tier 3 (General Inquiries)", "Email", "General_Standard / General_Lite", "Day 1, Day 7 / Day 14", "Inside Sales / Auto-Response"],
]

# Save to a local path in Colab
pdf_path = "Follow_Up_Workflow_Matrix.pdf"
doc = SimpleDocTemplate(pdf_path, pagesize=A4, title="Follow-Up Workflow Matrix")
styles = getSampleStyleSheet()

elements = [
    Paragraph("Indigifts — Follow-Up Workflow Matrix", styles['Heading1']),
    Spacer(1, 12),
    Paragraph("Concise mapping from Tier to channel, template and cadence. Use with the master spreadsheet.", styles['BodyText']),
    Spacer(1, 12)
]

table = Table(data, colWidths=[120, 140, 160, 120, 120])
table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#4F81BD")),
    ('TEXTCOLOR', (0,0), (-1,0), colors.white),
    ('ALIGN', (0,0), (-1,-1), 'CENTER'),
    ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
    ('FONTSIZE', (0,0), (-1,0), 10),
    ('BOTTOMPADDING', (0,0), (-1,0), 8),
    ('BACKGROUND', (0,1), (-1,-1), colors.whitesmoke),
    ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
    ('FONTSIZE', (0,1), (-1,-1), 9),
    ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
]))

elements.append(table)
doc.build(elements)

print(f"PDF saved to: {pdf_path}")

# Download the file
files.download(pdf_path)



PDF saved to: Follow_Up_Workflow_Matrix.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [173]:
!pip install streamlit pandas xlsxwriter phonenumbers python-docx reportlab pytz
!streamlit run app.py




2025-08-15 10:35:44.030 Port 8501 is already in use


In [194]:
# app.py — Indigifts Secretariat Dashboard (no emojis)

import io, os, re, zipfile
import datetime as dt
import pytz
import pandas as pd
import streamlit as st
import phonenumbers
from docx import Document
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet

# ---------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------
TIMEZONE = "Asia/Kolkata"
TODAY = dt.datetime.now(pytz.timezone(TIMEZONE)).date()
TODAY_DT = dt.datetime(TODAY.year, TODAY.month, TODAY.day)
CATEGORY_LIST = ["HNI", "Founder", "Retail", "Corporate"]
STATUS_LIST   = ["New", "Contacted", "Proposal Sent", "Closed"]
FOLLOW_SLOTS  = ["Day 1", "Day 2", "Day 3", "Day 5", "Day 7", "Day 14"]

st.set_page_config(page_title="Indigifts Secretariat", layout="wide", initial_sidebar_state="expanded")

# ---------------------------------------------------------------------
# Banner Image (Medium Size) on Top
# ---------------------------------------------------------------------
BANNER_PATH = "INDIGIFT.png"  # already in Colab working dir
if os.path.exists(BANNER_PATH):
    st.image(BANNER_PATH, width=500)  # medium size
else:
    st.markdown(
        "<div style='background-color:#f4f4f4;padding:20px;text-align:center;font-size:22px;'>"
        "Indigifts Secretariat Dashboard"
        "</div>", unsafe_allow_html=True
    )

# Optional caption under banner
st.caption(f"Timezone: {TIMEZONE} | Today: {TODAY}")



# ---------------------------------------------------------------------
# Helpers: cleansing, scoring, exports
# ---------------------------------------------------------------------
EXT_PAT = re.compile(r"(ext\.?|x)\s*\d+$", re.IGNORECASE)
EMAIL_RE = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
QTY_WORDS   = ("bulk", "large", "thousands", "hundreds")
URGENT_WORDS= ("urgent", "asap", "immediate", "this week", "rush", "priority")
BUDGET_PAT  = re.compile(r"(?:₹|rs\.?\s*)(\d[\d,\.]{2,})", re.IGNORECASE)
NUM_PAT     = re.compile(r"\b(\d{2,6})\b")

def pick(df, *cands):
    colmap = {c.lower(): c for c in df.columns}
    for c in cands:
        if c.lower() in colmap:
            return colmap[c.lower()]
    return None

def normalize_phone(x):
    s = str(x or "").strip()
    if len(re.sub(r"\D", "", s)) < 7:
        return s
    s = EXT_PAT.sub("", s)
    digits = re.sub(r"\D", "", s)
    if len(digits) == 10:
        digits = "91" + digits.lstrip("0")
    try:
        parsed = phonenumbers.parse("+" + digits, None)
        if phonenumbers.is_possible_number(parsed) and phonenumbers.is_valid_number(parsed):
            return phonenumbers.format_number(parsed, phonenumbers.PhoneNumberFormat.E164)
    except Exception:
        pass
    if 10 <= len(digits) <= 15:
        return "+" + digits
    return s

def assign_tier(cat):
    c = str(cat).strip()
    if c in ("HNI", "Founder"):
        return "Tier 1"
    if c in ("Retail", "Corporate"):
        return "Tier 2"
    return "Tier 3"

def base_by_tier(tier: str) -> int:
    t = str(tier).strip().lower()
    if t == "tier 1": return 4
    if t == "tier 2": return 3
    return 2

def parse_signals(text: str):
    txt = str(text or "").lower()
    qty = None
    for m in NUM_PAT.finditer(txt):
        try:
            n = int(m.group(1).replace(",", ""))
            qty = max(qty or 0, n)
        except:
            pass
    qty_flag = any(w in txt for w in QTY_WORDS) or (qty is not None and qty >= 200)

    bud = None
    bm = BUDGET_PAT.search(txt)
    if bm:
        raw = bm.group(1).replace(",", "")
        try:
            bud = float(raw)
        except:
            pass
    bud_flag = bud is not None and bud >= 100000
    urgent_flag = any(w in txt for w in URGENT_WORDS)
    return qty_flag, bud_flag, urgent_flag

def priority_score(row) -> int:
    score = base_by_tier(row.get("Tier", ""))
    qf, bf, uf = parse_signals(row.get("Query Details", ""))
    for flag in (qf, bf, uf):
        if flag:
            score += 1
    return int(min(max(score, 1), 5))

def infer_category(title, company, query):
    t = (str(title) + " " + str(company)).lower()
    q = str(query).lower()
    # Founder / HNI signals
    if any(k in t for k in ["founder", "co-founder", "ceo", "md", "managing director", "entrepreneur"]) \
       or any(k in t for k in ["investor", "angel", "hni", "vc"]):
        if ("investor" in t) or ("hni" in t) or ("investor" in q) or ("hni" in q):
            return "HNI"
        return "Founder"
    # Retail signals
    if any(k in t for k in ["retail", "store", "chain", "bazaar", "mart", "boutique", "shop", "market", "galleria"]):
        return "Retail"
    # Query text clues
    if any(k in q for k in ["wedding", "personal", "hni", "investor", "luxury"]):
        return "HNI"
    if any(k in q for k in ["retail", "shop", "store", "chain", "bazaar", "market"]):
        return "Retail"
    if any(k in q for k in ["corporate", "employee", "client", "company", "vendor"]):
        return "Corporate"
    # fallback
    return "Corporate"

def clean_and_enrich(df, cap_rows=None):
    # Canonical mapping
    first  = pick(df, "First Name", "First", "FirstName")
    last   = pick(df, "Last Name", "Last", "LastName")
    company= pick(df, "Company")
    title  = pick(df, "Job Title", "Title")
    phone1 = pick(df, "Phone 1", "Phone1", "Phone")
    email1 = pick(df, "Email 1", "Email1", "Email")
    notes  = pick(df, "Notes", "Description", "Remarks")

    if first and last:
        name_series = df[first].fillna("").astype(str).str.strip() + " " + df[last].fillna("").astype(str).str.strip()
    else:
        name = pick(df, "Name")
        name_series = df[name] if name else pd.Series([""]*len(df))

    canon = pd.DataFrame({
        "Name": name_series.str.strip(),
        "Company": df[company] if company else "",
        "Title": df[title] if title else "",
        "Mobile": df[phone1] if phone1 else "",
        "Email": (df[email1].astype(str).str.strip().str.lower() if email1 else ""),
        "Query Details": df[notes] if notes else "",
    })
    if cap_rows:
        canon = canon.head(int(cap_rows)).copy()

    # Dedupe
    canon["__key__"] = (
        canon["Name"].fillna("").str.lower().str.strip() + "|" +
        canon["Company"].fillna("").str.lower().str.strip() + "|" +
        canon["Email"].fillna("").str.lower().str.strip()
    )
    canon = canon.drop_duplicates("__key__").drop(columns="__key__")
    canon = (canon.sort_values(["Email", "Company"], kind="stable")
                  .drop_duplicates(subset=["Email"], keep="first"))

    # Clean
    canon["Mobile (E.164)"] = canon["Mobile"].apply(normalize_phone)
    canon["Email_Valid"] = canon["Email"].apply(lambda e: bool(EMAIL_RE.match(str(e).lower().strip())))
    if "Category" not in canon.columns:
        canon["Category"] = ""
    canon["Category"] = canon.apply(
        lambda r: (r.get("Category", "") if str(r.get("Category", "")).strip() != ""
                   else infer_category(r.get("Title", ""), r.get("Company", ""), r.get("Query Details", ""))),
        axis=1
    )
    canon["Tier"] = canon["Category"].apply(assign_tier)

    master = canon.copy()
    master["Status"] = "New"
    master["Follow-up Slot"] = ""
    master["Follow-up Date"] = TODAY
    master["Next Action"] = ""
    master["Priority Score (1-5)"] = master.apply(priority_score, axis=1)
    return master

def make_excel(master: pd.DataFrame) -> bytes:
    out = io.BytesIO()
    with pd.ExcelWriter(out, engine="xlsxwriter", datetime_format="yyyy-mm-dd", date_format="yyyy-mm-dd") as writer:
        master.to_excel(writer, sheet_name="Master", index=False)

        wb  = writer.book
        ws  = writer.sheets["Master"]

        # Lists sheet
        max_len = max(len(CATEGORY_LIST), len(STATUS_LIST), len(FOLLOW_SLOTS))
        def pad(lst, n): return lst + [""] * (n - len(lst))
        lists_df = pd.DataFrame({
            "Category": pad(CATEGORY_LIST, max_len),
            "Status": pad(STATUS_LIST, max_len),
            "Follow-up Slot": pad(FOLLOW_SLOTS, max_len)
        })
        lists_df.to_excel(writer, sheet_name="Lists", index=False)

        # Named ranges
        wb.define_name("CategoryList",   f'=Lists!$A$2:$A${1+len(CATEGORY_LIST)}')
        wb.define_name("StatusList",     f'=Lists!$B$2:$B${1+len(STATUS_LIST)}')
        wb.define_name("FollowSlotList", f'=Lists!$C$2:$C${1+len(FOLLOW_SLOTS)}')

        nrows = len(master)
        col_idx = {c:i for i,c in enumerate(master.columns)}

        # Dropdowns and date validation
        if "Category" in col_idx:
            ws.data_validation(1, col_idx["Category"], nrows, col_idx["Category"],
                               {'validate':'list','source':'=CategoryList'})
        if "Status" in col_idx:
            ws.data_validation(1, col_idx["Status"], nrows, col_idx["Status"],
                               {'validate':'list','source':'=StatusList'})
        if "Follow-up Slot" in col_idx:
            ws.data_validation(1, col_idx["Follow-up Slot"], nrows, col_idx["Follow-up Slot"],
                               {'validate':'list','source':'=FollowSlotList'})
        if "Follow-up Date" in col_idx:
            ws.data_validation(1, col_idx["Follow-up Date"], nrows, col_idx["Follow-up Date"],
                               {'validate':'date','criteria':'>=','value': TODAY_DT})

        # Conditional formatting on Priority
        if "Priority Score (1-5)" in master.columns:
            pcol = master.columns.get_loc("Priority Score (1-5)")
            fmt_high = wb.add_format({"bold": True})
            fmt_mid  = wb.add_format({})
            fmt_low  = wb.add_format({"italic": True})
            ws.conditional_format(1, pcol, nrows, pcol, {"type":"cell","criteria":"==","value":5,"format":fmt_high})
            ws.conditional_format(1, pcol, nrows, pcol, {"type":"cell","criteria":"==","value":4,"format":fmt_mid})
            ws.conditional_format(1, pcol, nrows, pcol, {"type":"cell","criteria":"<=","value":2,"format":fmt_low})

        # Highlights sheets
        (master.groupby("Category").size().rename("Count").to_frame()
               .to_excel(writer, sheet_name="By Category"))
        (master.groupby("Tier").size().rename("Count").to_frame()
               .to_excel(writer, sheet_name="By Tier"))
        (master.groupby("Status").size().rename("Count").to_frame()
               .to_excel(writer, sheet_name="By Status"))
        (master.groupby("Category")["Priority Score (1-5)"].mean().round(2)
               .rename("Avg Priority").to_frame()
               .to_excel(writer, sheet_name="Avg Priority by Category"))

        # README
        readme = f"""Indigifts Secretariat Master Sheet

- Mobile (E.164) standardization; 10-digit assumed India +91
- Email_Valid basic format check
- Category dropdown: HNI, Founder, Retail, Corporate
- Tier: Tier 1 (HNI/Founder), Tier 2 (Retail/Corporate), Tier 3 (Others)
- Priority Score (1–5): Tier base (T1=4, T2=3, T3=2) + signals (quantity, budget, urgency) capped at 5
- Follow-up Date must be ≥ {TODAY}
"""
        pd.DataFrame({"README": readme.split("\n")}).to_excel(writer, sheet_name="README", index=False)

    out.seek(0)
    return out.read()

def make_workflow_df():
    rows = []
    for tier in ["Tier 1", "Tier 2", "Tier 3"]:
        for score in range(1,6):
            if tier == "Tier 1":
                if score >= 4:
                    follow = "Day 1, Day 3, Day 7, Day 14"; channel="Call + Email"; owner="Leadership / Sr. Sales"; template="HNI_Premium"
                elif score >= 3:
                    follow = "Day 1, Day 5, Day 14"; channel="Call + Email"; owner="Sr. Sales"; template="HNI_Standard"
                else:
                    follow = "Day 1, Day 7"; channel="Email"; owner="Mid-level Sales"; template="HNI_Lite"
            elif tier == "Tier 2":
                if score >= 4:
                    follow = "Day 1, Day 5, Day 14"; channel="Call + Email"; owner="Mid-level Sales"; template="Corporate_Bulk"
                elif score >= 3:
                    follow = "Day 1, Day 7"; channel="Email"; owner="Mid-level Sales"; template="Corporate_Standard"
                else:
                    follow = "Day 1, Day 14"; channel="Email"; owner="Inside Sales"; template="Corporate_Lite"
            else:
                if score >= 4:
                    follow = "Day 1, Day 7"; channel="Email"; owner="Inside Sales"; template="General_Standard"
                elif score >= 3:
                    follow = "Day 1, Day 14"; channel="Email"; owner="Inside Sales"; template="General_Lite"
                else:
                    follow = "Day 1"; channel="Email"; owner="Auto-Response"; template="General_Auto"
            rows.append({
                "Tier": tier,
                "Priority Score (1-5)": score,
                "Follow-Up Slots": follow,
                "Channel": channel,
                "Owner": owner,
                "Proposal Template": template
            })
    return pd.DataFrame(rows)

def make_proposal_docx_tier1() -> bytes:
    doc = Document()
    doc.add_heading("Proposal Outline — Tier 1 (HNI / Founders)", level=1)
    doc.add_heading("Executive Summary", level=2)
    doc.add_paragraph("• Premium, personalized gifting for HNI/founder clientele.")
    doc.add_paragraph("• Focus on exclusivity, customization, white-glove fulfilment.")
    doc.add_heading("Proposed Solution", level=2)
    doc.add_paragraph("• Curated luxury sets; limited editions; bespoke packaging; handwritten notes.")
    doc.add_heading("Next Steps", level=2)
    doc.add_paragraph("• Schedule a 20-minute consult to confirm preferences and budget.")
    doc.add_paragraph("• Share mood boards and tiered pricing (₹1L–₹10L+).")
    doc.add_heading("CTA", level=2)
    doc.add_paragraph("• Reply with preferred call time or approve the curated shortlist.")
    buff = io.BytesIO(); doc.save(buff); buff.seek(0); return buff.read()

def make_proposal_docx_tier2() -> bytes:
    doc = Document()
    doc.add_heading("Proposal Outline — Tier 2 (Retail / Corporate)", level=1)

    doc.add_heading("Executive Summary", level=2)
    doc.add_paragraph("• Scalable corporate/retail gifting balancing quality, speed, and cost.")

    doc.add_heading("Proposed Solution", level=2)
    doc.add_paragraph("• Branded bestsellers with bulk pricing slabs.")
    doc.add_paragraph("• Kitting, address management, and PAN‑India delivery.")
    doc.add_paragraph("• SLA‑bound fulfilment with QC and replacement support.")

    doc.add_heading("Next Steps", level=2)
    doc.add_paragraph("• Share quantity tiers (e.g., 200/500/1000+), delivery cities, and budget bands.")
    doc.add_paragraph("• Confirm branding assets (logo, colour codes) and packaging preferences.")

    doc.add_heading("CTA", level=2)
    doc.add_paragraph("• Reply with quantities and branding files to receive a tiered quote today.")

    buf = io.BytesIO(); doc.save(buf); buf.seek(0); return buf.read()

def make_proposal_docx_tier3() -> bytes:
    doc = Document()
    doc.add_heading("Proposal Outline — Tier 3 (General Inquiries)", level=1)
    doc.add_heading("Executive Summary", level=2)
    doc.add_paragraph("• Fast, affordable options with dependable SLAs.")
    doc.add_heading("Proposed Solution", level=2)
    doc.add_paragraph("• Ready-to-ship catalog; bundle discounts; simple branding.")
    doc.add_heading("Next Steps", level=2)
    doc.add_paragraph("• Share quantity and delivery date; confirm address list template.")
    doc.add_heading("CTA", level=2)
    doc.add_paragraph("• Reply with quantity for a same-day quote.")
    buff = io.BytesIO(); doc.save(buff); buff.seek(0); return buff.read()

def make_workflow_pdf() -> bytes:
    buff = io.BytesIO()
    doc = SimpleDocTemplate(buff, pagesize=A4, title="Follow-Up Workflow Matrix")
    styles = getSampleStyleSheet()
    elements = [
        Paragraph("Indigifts — Follow-Up Workflow Matrix", styles['Heading1']),
        Spacer(1, 12),
        Paragraph("Mapping from Tier to channel, template and cadence. Use with the master spreadsheet.", styles['BodyText']),
        Spacer(1, 12)
    ]
    data = [
        ["Tier", "Communication Channel", "Template Name", "Timing", "Owner Role"],
        ["Tier 1 (HNIs/Founders)", "Call + Email", "HNI_Premium / HNI_Standard", "Day 1, Day 3, Day 7, Day 14", "Leadership / Sr. Sales"],
        ["Tier 2 (Retail/Corporate)", "Call + Email", "Corporate_Bulk / Corporate_Standard", "Day 1, Day 5, Day 14", "Mid-level Sales"],
        ["Tier 3 (General Inquiries)", "Email", "General_Standard / General_Lite", "Day 1, Day 7 / Day 14", "Inside Sales / Auto-Response"],
    ]
    table = Table(data, colWidths=[120, 140, 160, 120, 120])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#4F81BD")),
        ('TEXTCOLOR', (0,0), (-1,0), colors.white),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('FONTSIZE', (0,0), (-1,0), 10),
        ('BOTTOMPADDING', (0,0), (-1,0), 8),
        ('BACKGROUND', (0,1), (-1,-1), colors.whitesmoke),
        ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
        ('FONTSIZE', (0,1), (-1,-1), 9),
        ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
    ]))
    elements.append(table)
    doc.build(elements)
    buff.seek(0)
    return buff.read()

def make_zip(files_dict: dict) -> bytes:
    buff = io.BytesIO()
    with zipfile.ZipFile(buff, "w", zipfile.ZIP_DEFLATED) as zf:
        for fname, fbytes in files_dict.items():
            zf.writestr(fname, fbytes)
    buff.seek(0)
    return buff.read()

# ---------------------------------------------------------------------
# Layout
# ---------------------------------------------------------------------
with st.sidebar:
    st.markdown("Upload a raw leads CSV, review and filter the master list, then download deliverables.")
    cap_rows = st.number_input("Row limit (optional)", min_value=10, max_value=10000, value=50, step=10)
    st.markdown("---")
    st.markdown("Deliverables:")
    st.write("- Master Excel (.xlsx)")
    st.write("- Workflow PDF")
    st.write("- Proposal DOCX (Tier 1, Tier 2, Tier 3)")
    st.write("- ZIP bundle (all files)")

uploaded = st.file_uploader("Upload Raw Leads CSV", type=["csv"])

if uploaded:
    raw_df = pd.read_csv(uploaded)
    master = clean_and_enrich(raw_df, cap_rows=cap_rows)

    # Filters and search
    fcol1, fcol2, fcol3, fcol4 = st.columns([1,1,1,2])
    f_tier = fcol1.multiselect("Filter by Tier", ["Tier 1","Tier 2","Tier 3"], default=["Tier 1","Tier 2","Tier 3"])
    f_cat  = fcol2.multiselect("Filter by Category", CATEGORY_LIST, default=CATEGORY_LIST)
    f_minp = fcol3.slider("Minimum Priority", 1, 5, 1)
    q = fcol4.text_input("Search (Name/Company/Email contains)")

    view = master[
        master["Tier"].isin(f_tier) &
        master["Category"].isin(f_cat) &
        (master["Priority Score (1-5)"] >= f_minp)
    ].copy()

    if q.strip():
        ql = q.strip().lower()
        view = view[
            view["Name"].str.lower().str.contains(ql) |
            view["Company"].str.lower().str.contains(ql) |
            view["Email"].str.lower().str.contains(ql)
        ]

    st.markdown("### Master Lead List")
    st.dataframe(view, use_container_width=True, hide_index=True)

    # Metrics and simple charts
    m1, m2, m3, m4 = st.columns(4)
    m1.metric("Total Leads", len(master))
    m2.metric("Tier 1", int((master["Tier"]=="Tier 1").sum()))
    m3.metric("Tier 2", int((master["Tier"]=="Tier 2").sum()))
    m4.metric("Tier 3", int((master["Tier"]=="Tier 3").sum()))

    c1, c2 = st.columns(2)
    c1.bar_chart(master.groupby("Category").size())
    c2.bar_chart(master.groupby("Tier").size())

    # Workflow Matrix
    st.markdown("### Follow-Up Workflow Matrix")
    workflow_df = make_workflow_df()
    st.dataframe(workflow_df, use_container_width=True, hide_index=True)

    # Proposal Skeletons
    st.markdown("### Proposal Skeletons")
    p1, p2, p3 = st.columns(3)

    with p1:
        st.markdown("**Tier 1 – High Value (HNI/Founders)**")
        st.markdown("- Executive Summary: premium, personalized approach")
        st.markdown("- Proposed Solution: curated luxury sets, bespoke packaging, concierge logistics")
        st.markdown("- Next Steps: 20‑minute consult, mood boards, tiered pricing")
        st.markdown("- CTA: reply with preferred call time or approve shortlist")

    with p2:
        st.markdown("**Tier 2 – Retail / Corporate (Bulk)**")
        st.markdown("- Executive Summary: scalable, quality‑first, cost‑aware gifting")
        st.markdown("- Proposed Solution: branded bestsellers, bulk pricing, kitting, address management")
        st.markdown("- Next Steps: share quantity tiers, delivery cities, budget bands")
        st.markdown("- CTA: send quantities + branding files for a tiered quote")

    with p3:
        st.markdown("**Tier 3 – General Inquiry**")
        st.markdown("- Executive Summary: fast, dependable options")
        st.markdown("- Proposed Solution: ready‑to‑ship catalog, bundle discounts, simple branding")
        st.markdown("- Next Steps: share quantity and delivery date")
        st.markdown("- CTA: reply with quantity for a same‑day quote")

    # Automation note
    st.markdown("### Automation and Tooling Recommendation")
    st.write(
        "Use Google Sheets as the live master. A Zapier flow can append new CSV rows to the sheet, "
        "create Google Calendar reminders based on Follow-up Date, and notify the assigned owner via email. "
        "Optionally run a small Python script as a daily cron to send summary reminders for due and overdue follow-ups."
    )

    # Build deliverables
    excel_bytes = make_excel(master)
    pdf_bytes   = make_workflow_pdf()
    d1_bytes = make_proposal_docx_tier1()
    d2_bytes = make_proposal_docx_tier2()
    d3_bytes = make_proposal_docx_tier3()

    zip_bytes = make_zip({
        "indigifts_master_with_workflow.xlsx": excel_bytes,
        "Follow_Up_Workflow_Matrix.pdf": pdf_bytes,
        "proposal_tier1.docx": d1_bytes,
        "proposal_tier2.docx": d2_bytes,
        "proposal_tier3.docx": d3_bytes,
        "tooling_note.txt": (
            "Automation Recommendation:\n"
            "Google Sheets + Zapier: ingest rows; Calendar reminders; email alerts.\n"
            "Optional daily Python cron for due/overdue follow-ups.\n"
        ).encode("utf-8")
    })

    st.markdown("---")
    dl1, dl2, dl3, dl4, dl5, dl6 = st.columns(6)

    with dl1:
        st.download_button("Download Master Excel", excel_bytes,
            file_name="indigifts_master_with_workflow.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            use_container_width=True)

    with dl2:
        st.download_button("Download Workflow PDF", pdf_bytes,
            file_name="Follow_Up_Workflow_Matrix.pdf",
            mime="application/pdf",
            use_container_width=True)

    with dl3:
        st.download_button("Download Tier 1 Proposal", d1_bytes,
            file_name="proposal_tier1.docx",
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
            use_container_width=True)

    with dl4:
        st.download_button("Download Tier 2 Proposal", d2_bytes,
            file_name="proposal_tier2.docx",
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
            use_container_width=True)

    with dl5:
        st.download_button("Download Tier 3 Proposal", d3_bytes,
            file_name="proposal_tier3.docx",
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
            use_container_width=True)

    with dl6:
        st.download_button("Download ZIP (All)", zip_bytes,
            file_name="Indigifts_Practice_Exercise_Submission.zip",
            mime="application/zip",
            use_container_width=True)

else:
    st.info("Upload a CSV file to start.")


2025-08-15 10:47:07.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 10:47:07.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 10:47:07.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 10:47:07.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 10:47:07.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 10:47:07.858 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 10:47:07.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 10:47:07.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [175]:
 !pip install pyngrok


In [176]:
!pip -q install streamlit pyngrok pandas xlsxwriter phonenumbers python-docx reportlab pytz


In [196]:
# Cell 3 — write app.py
app_code = r'''

# app.py — Indigifts Secretariat Dashboard (no emojis)

import io, os, re, zipfile
import datetime as dt
import pytz
import pandas as pd
import streamlit as st
import phonenumbers
from docx import Document
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet

# ---------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------
TIMEZONE = "Asia/Kolkata"
TODAY = dt.datetime.now(pytz.timezone(TIMEZONE)).date()
TODAY_DT = dt.datetime(TODAY.year, TODAY.month, TODAY.day)
CATEGORY_LIST = ["HNI", "Founder", "Retail", "Corporate"]
STATUS_LIST   = ["New", "Contacted", "Proposal Sent", "Closed"]
FOLLOW_SLOTS  = ["Day 1", "Day 2", "Day 3", "Day 5", "Day 7", "Day 14"]

st.set_page_config(page_title="Indigifts Secretariat", layout="wide", initial_sidebar_state="expanded")

# ---------------------------------------------------------------------
# Banner Image (Medium Size) on Top
# ---------------------------------------------------------------------
BANNER_PATH = "INDIGIFT.png"  # already in Colab working dir
if os.path.exists(BANNER_PATH):
    st.image(BANNER_PATH, width=500)  # medium size
else:
    st.markdown(
        "<div style='background-color:#f4f4f4;padding:20px;text-align:center;font-size:22px;'>"
        "Indigifts Secretariat Dashboard"
        "</div>", unsafe_allow_html=True
    )

# Optional caption under banner
st.caption(f"Timezone: {TIMEZONE} | Today: {TODAY}")



# ---------------------------------------------------------------------
# Helpers: cleansing, scoring, exports
# ---------------------------------------------------------------------
EXT_PAT = re.compile(r"(ext\.?|x)\s*\d+$", re.IGNORECASE)
EMAIL_RE = re.compile(r"^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")
QTY_WORDS   = ("bulk", "large", "thousands", "hundreds")
URGENT_WORDS= ("urgent", "asap", "immediate", "this week", "rush", "priority")
BUDGET_PAT  = re.compile(r"(?:₹|rs\.?\s*)(\d[\d,\.]{2,})", re.IGNORECASE)
NUM_PAT     = re.compile(r"\b(\d{2,6})\b")

def pick(df, *cands):
    colmap = {c.lower(): c for c in df.columns}
    for c in cands:
        if c.lower() in colmap:
            return colmap[c.lower()]
    return None

def normalize_phone(x):
    s = str(x or "").strip()
    if len(re.sub(r"\D", "", s)) < 7:
        return s
    s = EXT_PAT.sub("", s)
    digits = re.sub(r"\D", "", s)
    if len(digits) == 10:
        digits = "91" + digits.lstrip("0")
    try:
        parsed = phonenumbers.parse("+" + digits, None)
        if phonenumbers.is_possible_number(parsed) and phonenumbers.is_valid_number(parsed):
            return phonenumbers.format_number(parsed, phonenumbers.PhoneNumberFormat.E164)
    except Exception:
        pass
    if 10 <= len(digits) <= 15:
        return "+" + digits
    return s

def assign_tier(cat):
    c = str(cat).strip()
    if c in ("HNI", "Founder"):
        return "Tier 1"
    if c in ("Retail", "Corporate"):
        return "Tier 2"
    return "Tier 3"

def base_by_tier(tier: str) -> int:
    t = str(tier).strip().lower()
    if t == "tier 1": return 4
    if t == "tier 2": return 3
    return 2

def parse_signals(text: str):
    txt = str(text or "").lower()
    qty = None
    for m in NUM_PAT.finditer(txt):
        try:
            n = int(m.group(1).replace(",", ""))
            qty = max(qty or 0, n)
        except:
            pass
    qty_flag = any(w in txt for w in QTY_WORDS) or (qty is not None and qty >= 200)

    bud = None
    bm = BUDGET_PAT.search(txt)
    if bm:
        raw = bm.group(1).replace(",", "")
        try:
            bud = float(raw)
        except:
            pass
    bud_flag = bud is not None and bud >= 100000
    urgent_flag = any(w in txt for w in URGENT_WORDS)
    return qty_flag, bud_flag, urgent_flag

def priority_score(row) -> int:
    score = base_by_tier(row.get("Tier", ""))
    qf, bf, uf = parse_signals(row.get("Query Details", ""))
    for flag in (qf, bf, uf):
        if flag:
            score += 1
    return int(min(max(score, 1), 5))

def infer_category(title, company, query):
    t = (str(title) + " " + str(company)).lower()
    q = str(query).lower()
    # Founder / HNI signals
    if any(k in t for k in ["founder", "co-founder", "ceo", "md", "managing director", "entrepreneur"]) \
       or any(k in t for k in ["investor", "angel", "hni", "vc"]):
        if ("investor" in t) or ("hni" in t) or ("investor" in q) or ("hni" in q):
            return "HNI"
        return "Founder"
    # Retail signals
    if any(k in t for k in ["retail", "store", "chain", "bazaar", "mart", "boutique", "shop", "market", "galleria"]):
        return "Retail"
    # Query text clues
    if any(k in q for k in ["wedding", "personal", "hni", "investor", "luxury"]):
        return "HNI"
    if any(k in q for k in ["retail", "shop", "store", "chain", "bazaar", "market"]):
        return "Retail"
    if any(k in q for k in ["corporate", "employee", "client", "company", "vendor"]):
        return "Corporate"
    # fallback
    return "Corporate"

def clean_and_enrich(df, cap_rows=None):
    # Canonical mapping
    first  = pick(df, "First Name", "First", "FirstName")
    last   = pick(df, "Last Name", "Last", "LastName")
    company= pick(df, "Company")
    title  = pick(df, "Job Title", "Title")
    phone1 = pick(df, "Phone 1", "Phone1", "Phone")
    email1 = pick(df, "Email 1", "Email1", "Email")
    notes  = pick(df, "Notes", "Description", "Remarks")

    if first and last:
        name_series = df[first].fillna("").astype(str).str.strip() + " " + df[last].fillna("").astype(str).str.strip()
    else:
        name = pick(df, "Name")
        name_series = df[name] if name else pd.Series([""]*len(df))

    canon = pd.DataFrame({
        "Name": name_series.str.strip(),
        "Company": df[company] if company else "",
        "Title": df[title] if title else "",
        "Mobile": df[phone1] if phone1 else "",
        "Email": (df[email1].astype(str).str.strip().str.lower() if email1 else ""),
        "Query Details": df[notes] if notes else "",
    })
    if cap_rows:
        canon = canon.head(int(cap_rows)).copy()

    # Dedupe
    canon["__key__"] = (
        canon["Name"].fillna("").str.lower().str.strip() + "|" +
        canon["Company"].fillna("").str.lower().str.strip() + "|" +
        canon["Email"].fillna("").str.lower().str.strip()
    )
    canon = canon.drop_duplicates("__key__").drop(columns="__key__")
    canon = (canon.sort_values(["Email", "Company"], kind="stable")
                  .drop_duplicates(subset=["Email"], keep="first"))

    # Clean
    canon["Mobile (E.164)"] = canon["Mobile"].apply(normalize_phone)
    canon["Email_Valid"] = canon["Email"].apply(lambda e: bool(EMAIL_RE.match(str(e).lower().strip())))
    if "Category" not in canon.columns:
        canon["Category"] = ""
    canon["Category"] = canon.apply(
        lambda r: (r.get("Category", "") if str(r.get("Category", "")).strip() != ""
                   else infer_category(r.get("Title", ""), r.get("Company", ""), r.get("Query Details", ""))),
        axis=1
    )
    canon["Tier"] = canon["Category"].apply(assign_tier)

    master = canon.copy()
    master["Status"] = "New"
    master["Follow-up Slot"] = ""
    master["Follow-up Date"] = TODAY
    master["Next Action"] = ""
    master["Priority Score (1-5)"] = master.apply(priority_score, axis=1)
    return master

def make_excel(master: pd.DataFrame) -> bytes:
    out = io.BytesIO()
    with pd.ExcelWriter(out, engine="xlsxwriter", datetime_format="yyyy-mm-dd", date_format="yyyy-mm-dd") as writer:
        master.to_excel(writer, sheet_name="Master", index=False)

        wb  = writer.book
        ws  = writer.sheets["Master"]

        # Lists sheet
        max_len = max(len(CATEGORY_LIST), len(STATUS_LIST), len(FOLLOW_SLOTS))
        def pad(lst, n): return lst + [""] * (n - len(lst))
        lists_df = pd.DataFrame({
            "Category": pad(CATEGORY_LIST, max_len),
            "Status": pad(STATUS_LIST, max_len),
            "Follow-up Slot": pad(FOLLOW_SLOTS, max_len)
        })
        lists_df.to_excel(writer, sheet_name="Lists", index=False)

        # Named ranges
        wb.define_name("CategoryList",   f'=Lists!$A$2:$A${1+len(CATEGORY_LIST)}')
        wb.define_name("StatusList",     f'=Lists!$B$2:$B${1+len(STATUS_LIST)}')
        wb.define_name("FollowSlotList", f'=Lists!$C$2:$C${1+len(FOLLOW_SLOTS)}')

        nrows = len(master)
        col_idx = {c:i for i,c in enumerate(master.columns)}

        # Dropdowns and date validation
        if "Category" in col_idx:
            ws.data_validation(1, col_idx["Category"], nrows, col_idx["Category"],
                               {'validate':'list','source':'=CategoryList'})
        if "Status" in col_idx:
            ws.data_validation(1, col_idx["Status"], nrows, col_idx["Status"],
                               {'validate':'list','source':'=StatusList'})
        if "Follow-up Slot" in col_idx:
            ws.data_validation(1, col_idx["Follow-up Slot"], nrows, col_idx["Follow-up Slot"],
                               {'validate':'list','source':'=FollowSlotList'})
        if "Follow-up Date" in col_idx:
            ws.data_validation(1, col_idx["Follow-up Date"], nrows, col_idx["Follow-up Date"],
                               {'validate':'date','criteria':'>=','value': TODAY_DT})

        # Conditional formatting on Priority
        if "Priority Score (1-5)" in master.columns:
            pcol = master.columns.get_loc("Priority Score (1-5)")
            fmt_high = wb.add_format({"bold": True})
            fmt_mid  = wb.add_format({})
            fmt_low  = wb.add_format({"italic": True})
            ws.conditional_format(1, pcol, nrows, pcol, {"type":"cell","criteria":"==","value":5,"format":fmt_high})
            ws.conditional_format(1, pcol, nrows, pcol, {"type":"cell","criteria":"==","value":4,"format":fmt_mid})
            ws.conditional_format(1, pcol, nrows, pcol, {"type":"cell","criteria":"<=","value":2,"format":fmt_low})

        # Highlights sheets
        (master.groupby("Category").size().rename("Count").to_frame()
               .to_excel(writer, sheet_name="By Category"))
        (master.groupby("Tier").size().rename("Count").to_frame()
               .to_excel(writer, sheet_name="By Tier"))
        (master.groupby("Status").size().rename("Count").to_frame()
               .to_excel(writer, sheet_name="By Status"))
        (master.groupby("Category")["Priority Score (1-5)"].mean().round(2)
               .rename("Avg Priority").to_frame()
               .to_excel(writer, sheet_name="Avg Priority by Category"))

        # README
        readme = f"""Indigifts Secretariat Master Sheet

- Mobile (E.164) standardization; 10-digit assumed India +91
- Email_Valid basic format check
- Category dropdown: HNI, Founder, Retail, Corporate
- Tier: Tier 1 (HNI/Founder), Tier 2 (Retail/Corporate), Tier 3 (Others)
- Priority Score (1–5): Tier base (T1=4, T2=3, T3=2) + signals (quantity, budget, urgency) capped at 5
- Follow-up Date must be ≥ {TODAY}
"""
        pd.DataFrame({"README": readme.split("\n")}).to_excel(writer, sheet_name="README", index=False)

    out.seek(0)
    return out.read()

def make_workflow_df():
    rows = []
    for tier in ["Tier 1", "Tier 2", "Tier 3"]:
        for score in range(1,6):
            if tier == "Tier 1":
                if score >= 4:
                    follow = "Day 1, Day 3, Day 7, Day 14"; channel="Call + Email"; owner="Leadership / Sr. Sales"; template="HNI_Premium"
                elif score >= 3:
                    follow = "Day 1, Day 5, Day 14"; channel="Call + Email"; owner="Sr. Sales"; template="HNI_Standard"
                else:
                    follow = "Day 1, Day 7"; channel="Email"; owner="Mid-level Sales"; template="HNI_Lite"
            elif tier == "Tier 2":
                if score >= 4:
                    follow = "Day 1, Day 5, Day 14"; channel="Call + Email"; owner="Mid-level Sales"; template="Corporate_Bulk"
                elif score >= 3:
                    follow = "Day 1, Day 7"; channel="Email"; owner="Mid-level Sales"; template="Corporate_Standard"
                else:
                    follow = "Day 1, Day 14"; channel="Email"; owner="Inside Sales"; template="Corporate_Lite"
            else:
                if score >= 4:
                    follow = "Day 1, Day 7"; channel="Email"; owner="Inside Sales"; template="General_Standard"
                elif score >= 3:
                    follow = "Day 1, Day 14"; channel="Email"; owner="Inside Sales"; template="General_Lite"
                else:
                    follow = "Day 1"; channel="Email"; owner="Auto-Response"; template="General_Auto"
            rows.append({
                "Tier": tier,
                "Priority Score (1-5)": score,
                "Follow-Up Slots": follow,
                "Channel": channel,
                "Owner": owner,
                "Proposal Template": template
            })
    return pd.DataFrame(rows)

def make_proposal_docx_tier1() -> bytes:
    doc = Document()
    doc.add_heading("Proposal Outline — Tier 1 (HNI / Founders)", level=1)
    doc.add_heading("Executive Summary", level=2)
    doc.add_paragraph("• Premium, personalized gifting for HNI/founder clientele.")
    doc.add_paragraph("• Focus on exclusivity, customization, white-glove fulfilment.")
    doc.add_heading("Proposed Solution", level=2)
    doc.add_paragraph("• Curated luxury sets; limited editions; bespoke packaging; handwritten notes.")
    doc.add_heading("Next Steps", level=2)
    doc.add_paragraph("• Schedule a 20-minute consult to confirm preferences and budget.")
    doc.add_paragraph("• Share mood boards and tiered pricing (₹1L–₹10L+).")
    doc.add_heading("CTA", level=2)
    doc.add_paragraph("• Reply with preferred call time or approve the curated shortlist.")
    buff = io.BytesIO(); doc.save(buff); buff.seek(0); return buff.read()

def make_proposal_docx_tier2() -> bytes:
    doc = Document()
    doc.add_heading("Proposal Outline — Tier 2 (Retail / Corporate)", level=1)

    doc.add_heading("Executive Summary", level=2)
    doc.add_paragraph("• Scalable corporate/retail gifting balancing quality, speed, and cost.")

    doc.add_heading("Proposed Solution", level=2)
    doc.add_paragraph("• Branded bestsellers with bulk pricing slabs.")
    doc.add_paragraph("• Kitting, address management, and PAN‑India delivery.")
    doc.add_paragraph("• SLA‑bound fulfilment with QC and replacement support.")

    doc.add_heading("Next Steps", level=2)
    doc.add_paragraph("• Share quantity tiers (e.g., 200/500/1000+), delivery cities, and budget bands.")
    doc.add_paragraph("• Confirm branding assets (logo, colour codes) and packaging preferences.")

    doc.add_heading("CTA", level=2)
    doc.add_paragraph("• Reply with quantities and branding files to receive a tiered quote today.")

    buf = io.BytesIO(); doc.save(buf); buf.seek(0); return buf.read()

def make_proposal_docx_tier3() -> bytes:
    doc = Document()
    doc.add_heading("Proposal Outline — Tier 3 (General Inquiries)", level=1)
    doc.add_heading("Executive Summary", level=2)
    doc.add_paragraph("• Fast, affordable options with dependable SLAs.")
    doc.add_heading("Proposed Solution", level=2)
    doc.add_paragraph("• Ready-to-ship catalog; bundle discounts; simple branding.")
    doc.add_heading("Next Steps", level=2)
    doc.add_paragraph("• Share quantity and delivery date; confirm address list template.")
    doc.add_heading("CTA", level=2)
    doc.add_paragraph("• Reply with quantity for a same-day quote.")
    buff = io.BytesIO(); doc.save(buff); buff.seek(0); return buff.read()

def make_workflow_pdf() -> bytes:
    buff = io.BytesIO()
    doc = SimpleDocTemplate(buff, pagesize=A4, title="Follow-Up Workflow Matrix")
    styles = getSampleStyleSheet()
    elements = [
        Paragraph("Indigifts — Follow-Up Workflow Matrix", styles['Heading1']),
        Spacer(1, 12),
        Paragraph("Mapping from Tier to channel, template and cadence. Use with the master spreadsheet.", styles['BodyText']),
        Spacer(1, 12)
    ]
    data = [
        ["Tier", "Communication Channel", "Template Name", "Timing", "Owner Role"],
        ["Tier 1 (HNIs/Founders)", "Call + Email", "HNI_Premium / HNI_Standard", "Day 1, Day 3, Day 7, Day 14", "Leadership / Sr. Sales"],
        ["Tier 2 (Retail/Corporate)", "Call + Email", "Corporate_Bulk / Corporate_Standard", "Day 1, Day 5, Day 14", "Mid-level Sales"],
        ["Tier 3 (General Inquiries)", "Email", "General_Standard / General_Lite", "Day 1, Day 7 / Day 14", "Inside Sales / Auto-Response"],
    ]
    table = Table(data, colWidths=[120, 140, 160, 120, 120])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#4F81BD")),
        ('TEXTCOLOR', (0,0), (-1,0), colors.white),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('FONTSIZE', (0,0), (-1,0), 10),
        ('BOTTOMPADDING', (0,0), (-1,0), 8),
        ('BACKGROUND', (0,1), (-1,-1), colors.whitesmoke),
        ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
        ('FONTSIZE', (0,1), (-1,-1), 9),
        ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
    ]))
    elements.append(table)
    doc.build(elements)
    buff.seek(0)
    return buff.read()

def make_zip(files_dict: dict) -> bytes:
    buff = io.BytesIO()
    with zipfile.ZipFile(buff, "w", zipfile.ZIP_DEFLATED) as zf:
        for fname, fbytes in files_dict.items():
            zf.writestr(fname, fbytes)
    buff.seek(0)
    return buff.read()

# ---------------------------------------------------------------------
# Layout
# ---------------------------------------------------------------------
with st.sidebar:
    st.markdown("Upload a raw leads CSV, review and filter the master list, then download deliverables.")
    cap_rows = st.number_input("Row limit (optional)", min_value=10, max_value=10000, value=50, step=10)
    st.markdown("---")
    st.markdown("Deliverables:")
    st.write("- Master Excel (.xlsx)")
    st.write("- Workflow PDF")
    st.write("- Proposal DOCX (Tier 1, Tier 2, Tier 3)")
    st.write("- ZIP bundle (all files)")

uploaded = st.file_uploader("Upload Raw Leads CSV", type=["csv"])

if uploaded:
    raw_df = pd.read_csv(uploaded)
    master = clean_and_enrich(raw_df, cap_rows=cap_rows)

    # Filters and search
    fcol1, fcol2, fcol3, fcol4 = st.columns([1,1,1,2])
    f_tier = fcol1.multiselect("Filter by Tier", ["Tier 1","Tier 2","Tier 3"], default=["Tier 1","Tier 2","Tier 3"])
    f_cat  = fcol2.multiselect("Filter by Category", CATEGORY_LIST, default=CATEGORY_LIST)
    f_minp = fcol3.slider("Minimum Priority", 1, 5, 1)
    q = fcol4.text_input("Search (Name/Company/Email contains)")

    view = master[
        master["Tier"].isin(f_tier) &
        master["Category"].isin(f_cat) &
        (master["Priority Score (1-5)"] >= f_minp)
    ].copy()

    if q.strip():
        ql = q.strip().lower()
        view = view[
            view["Name"].str.lower().str.contains(ql) |
            view["Company"].str.lower().str.contains(ql) |
            view["Email"].str.lower().str.contains(ql)
        ]

    st.markdown("### Master Lead List")
    st.dataframe(view, use_container_width=True, hide_index=True)

    # Metrics and simple charts
    m1, m2, m3, m4 = st.columns(4)
    m1.metric("Total Leads", len(master))
    m2.metric("Tier 1", int((master["Tier"]=="Tier 1").sum()))
    m3.metric("Tier 2", int((master["Tier"]=="Tier 2").sum()))
    m4.metric("Tier 3", int((master["Tier"]=="Tier 3").sum()))

    c1, c2 = st.columns(2)
    c1.bar_chart(master.groupby("Category").size())
    c2.bar_chart(master.groupby("Tier").size())

    # Workflow Matrix
    st.markdown("### Follow-Up Workflow Matrix")
    workflow_df = make_workflow_df()
    st.dataframe(workflow_df, use_container_width=True, hide_index=True)

    # Proposal Skeletons
    st.markdown("### Proposal Skeletons")
    p1, p2, p3 = st.columns(3)

    with p1:
        st.markdown("**Tier 1 – High Value (HNI/Founders)**")
        st.markdown("- Executive Summary: premium, personalized approach")
        st.markdown("- Proposed Solution: curated luxury sets, bespoke packaging, concierge logistics")
        st.markdown("- Next Steps: 20‑minute consult, mood boards, tiered pricing")
        st.markdown("- CTA: reply with preferred call time or approve shortlist")

    with p2:
        st.markdown("**Tier 2 – Retail / Corporate (Bulk)**")
        st.markdown("- Executive Summary: scalable, quality‑first, cost‑aware gifting")
        st.markdown("- Proposed Solution: branded bestsellers, bulk pricing, kitting, address management")
        st.markdown("- Next Steps: share quantity tiers, delivery cities, budget bands")
        st.markdown("- CTA: send quantities + branding files for a tiered quote")

    with p3:
        st.markdown("**Tier 3 – General Inquiry**")
        st.markdown("- Executive Summary: fast, dependable options")
        st.markdown("- Proposed Solution: ready‑to‑ship catalog, bundle discounts, simple branding")
        st.markdown("- Next Steps: share quantity and delivery date")
        st.markdown("- CTA: reply with quantity for a same‑day quote")

    # Automation note
    st.markdown("### Automation and Tooling Recommendation")
    st.write(
        "Use Google Sheets as the live master. A Zapier flow can append new CSV rows to the sheet, "
        "create Google Calendar reminders based on Follow-up Date, and notify the assigned owner via email. "
        "Optionally run a small Python script as a daily cron to send summary reminders for due and overdue follow-ups."
    )

    # Build deliverables
    excel_bytes = make_excel(master)
    pdf_bytes   = make_workflow_pdf()
    d1_bytes = make_proposal_docx_tier1()
    d2_bytes = make_proposal_docx_tier2()
    d3_bytes = make_proposal_docx_tier3()

    zip_bytes = make_zip({
        "indigifts_master_with_workflow.xlsx": excel_bytes,
        "Follow_Up_Workflow_Matrix.pdf": pdf_bytes,
        "proposal_tier1.docx": d1_bytes,
        "proposal_tier2.docx": d2_bytes,
        "proposal_tier3.docx": d3_bytes,
        "tooling_note.txt": (
            "Automation Recommendation:\n"
            "Google Sheets + Zapier: ingest rows; Calendar reminders; email alerts.\n"
            "Optional daily Python cron for due/overdue follow-ups.\n"
        ).encode("utf-8")
    })

    st.markdown("---")
    dl1, dl2, dl3, dl4, dl5, dl6 = st.columns(6)

    with dl1:
        st.download_button("Download Master Excel", excel_bytes,
            file_name="indigifts_master_with_workflow.xlsx",
            mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            use_container_width=True)

    with dl2:
        st.download_button("Download Workflow PDF", pdf_bytes,
            file_name="Follow_Up_Workflow_Matrix.pdf",
            mime="application/pdf",
            use_container_width=True)

    with dl3:
        st.download_button("Download Tier 1 Proposal", d1_bytes,
            file_name="proposal_tier1.docx",
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
            use_container_width=True)

    with dl4:
        st.download_button("Download Tier 2 Proposal", d2_bytes,
            file_name="proposal_tier2.docx",
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
            use_container_width=True)

    with dl5:
        st.download_button("Download Tier 3 Proposal", d3_bytes,
            file_name="proposal_tier3.docx",
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
            use_container_width=True)

    with dl6:
        st.download_button("Download ZIP (All)", zip_bytes,
            file_name="Indigifts_Practice_Exercise_Submission.zip",
            mime="application/zip",
            use_container_width=True)

else:
    st.info("Upload a CSV file to start.")


'''
open("app.py","w",encoding="utf-8").write(app_code)
print("Wrote app.py")


Wrote app.py


In [197]:
# Kill anything using port 8501 (Streamlit)
!fuser -k 8501/tcp || true

# Close ALL ngrok tunnels and kill the ngrok agent
from pyngrok import ngrok, process

# Disconnect any active tunnels
try:
    for t in ngrok.get_tunnels():
        ngrok.disconnect(t.public_url)
        print("Disconnected:", t.public_url)
except Exception as e:
    print("No tunnels or error while listing:", repr(e))

# Kill the ngrok agent process (if running)
try:
    p = process.get_ngrok_process()
    p.proc.terminate()
    print("Terminated ngrok process.")
except Exception as e:
    print("No ngrok process to terminate:", repr(e))


8501/tcp:            112708


Disconnected: https://014ad989b78b.ngrok-free.app
No ngrok process to terminate: AttributeError("module 'pyngrok.process' has no attribute 'get_ngrok_process'")


In [198]:
# Cell A — write .streamlit/config.toml without bash
import os, textwrap, pathlib

pathlib.Path("/content/.streamlit").mkdir(parents=True, exist_ok=True)
cfg = textwrap.dedent("""
[server]
headless = true
enableCORS = false
enableXsrfProtection = false
port = 8501
address = "0.0.0.0"

[browser]
gatherUsageStats = false
""").strip()

with open("/content/.streamlit/config.toml", "w", encoding="utf-8") as f:
    f.write(cfg)

print("Wrote /content/.streamlit/config.toml ✓")


Wrote /content/.streamlit/config.toml ✓


In [199]:
%%bash
mkdir -p /content/.streamlit
cat > /content/.streamlit/config.toml <<'CFG'
[server]
headless = true
enableCORS = false
enableXsrfProtection = false
port = 8501
address = "0.0.0.0"

[browser]
gatherUsageStats = false
CFG
echo "Wrote /content/.streamlit/config.toml ✓"


Wrote /content/.streamlit/config.toml ✓


In [200]:
# Cell B — free the port and stop all ngrok tunnels cleanly
!fuser -k 8501/tcp || true

from pyngrok import ngrok

# Disconnect any existing tunnels
for t in ngrok.get_tunnels():
    try:
        ngrok.disconnect(t.public_url)
        print("Disconnected:", t.public_url)
    except Exception as e:
        print("Skip disconnect:", repr(e))

# Ensure the agent is not lingering
try:
    ngrok.kill()
    print("Killed ngrok agent.")
except Exception as e:
    print("No ngrok agent to kill:", repr(e))


Killed ngrok agent.


In [201]:
import os, subprocess, time, socket, sys

assert os.path.exists("/content/app.py"), "app.py not found at /content/app.py"

LOG = "/content/streamlit.log"

# Start Streamlit in the background (fresh)
proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port=8501", "--server.headless=true", "--server.address=0.0.0.0"],
    cwd="/content",
    stdout=open(LOG,"w"),
    stderr=subprocess.STDOUT,
    env={**os.environ, "PYTHONUNBUFFERED":"1"}
)
print("Starting Streamlit… PID:", proc.pid)

# Wait for the port to accept connections (up to ~25s)
def port_ready(host="127.0.0.1", port=8501, timeout=25):
    deadline = time.time() + timeout
    while time.time() < deadline:
        s = socket.socket()
        s.settimeout(1)
        try:
            s.connect((host, port))
            s.close()
            return True
        except Exception:
            time.sleep(1)
    return False

ok = port_ready()
print("Streamlit listening on 8501:", ok)
if not ok:
    print("---- Last 80 lines of streamlit.log ----")
    try:
        print("\n".join(open(LOG).read().splitlines()[-80:]))
    except Exception as e:
        print("Could not read log:", e)
    raise SystemExit("Streamlit failed to start. See log above.")


Starting Streamlit… PID: 114605
Streamlit listening on 8501: True


In [202]:
from pyngrok import ngrok

# If you haven't set your token in this runtime:
# ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN")

tunnels = ngrok.get_tunnels()
if tunnels:
    public_url = tunnels[0].public_url
else:
    public_url = ngrok.connect(addr=8501, proto="http").public_url

print("Public URL:", public_url)



Public URL: https://8a7d378d920e.ngrok-free.app


In [203]:
!tail -n +1 /content/streamlit.log



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

